<a href="https://colab.research.google.com/github/innovationcopilot/Chain-of-ThoughtsPapers/blob/main/0_BHIxTRF_ALL_NOTEBOOKS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. INITIAL SETUP AND ANALYSIS (CEO)

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
!pip install pycountry
import pycountry

# Generate a list of country names
country_names = [country.name for country in pycountry.countries]

# Load the CSV data
file_path = '/content/master_people_data.csv'  # Adjust the file path as necessary
data = pd.read_csv(file_path, encoding='latin1')

def identify_gender(name):
    name_str = str(name)
    return 'M' if 'Mr.' in name_str else 'F' if 'Mrs.' in name_str or 'Ms.' in name_str else 'N/A'

# Rename the 'PERMID' column to 'permid' for consistency
data.rename(columns={'co_id': 'ciq_id'}, inplace=True)

# Apply the identify_gender function to the 'Name' column
data['Gender'] = data['Name'].apply(identify_gender)

# Handle missing age values
data['Age'] = data['Age'].replace('--', 'N/A').fillna('N/A')

# Assign unique IDs to individuals based on their name and age
data['person_id'] = data.groupby(['Name', 'Age']).ngroup()  # Updated this line

# Step 1: Define Regular Expression for CEO Identification
ceo_regex = re.compile(r'\b(?:Chief Executive Officer|CEO)\b', re.IGNORECASE)

# Step 2: Apply the Regular Expression to Flag CEOs
data['CEO_Flag'] = data['Position'].apply(lambda x: bool(ceo_regex.search(str(x)))).astype(int)

# Function to check for 'group' or any country name in the 'Position' column
def check_position(position):
    if 'group' in position.lower():
        return 0
    if any(country.lower() in position.lower() for country in country_names):
        return 0
    return 1

# Apply the function to update the 'CEO_Flag' column
data['CEO_Flag'] = data['Position'].apply(lambda x: check_position(str(x))) * data['CEO_Flag']

# Step 3: Sort Data
data_sorted = data.sort_values(by=['CEO_Flag','person_id', 'Start_Year'])

# Calculate the previous job title based on unique person_id
data_sorted['Prev_Title'] = data_sorted.groupby('person_id')['Position'].shift()

# Calculate the officer start year
officer_start_year = data_sorted.groupby(['person_id', 'ciq_id'])['Start_Year'].min().reset_index()
officer_start_year.columns = ['person_id', 'ciq_id', 'Officer_Start_Year']

# Merge the officer start year data with the main data
data_merged = pd.merge(data_sorted, officer_start_year, on=['person_id', 'ciq_id'], how='left')

def extract_info(data):
    # Initialize an empty list to store the extracted information
    info_list = []

    col_list = ['permid', 'ciq_id', 'company', 'person_id', 'CEO_Flag', 'ceo', 'ceo_title', 'start_year', 'start_mo', 'end_year', 'end_mo', 'prev_job_id', 'prev_title', 'officer_start_yr', 'director_start_yr', 'gender', 'age', 'affiliations']

    # Iterate through the rows of the CSV data
    for index, row in data.iterrows():
        # Create a dictionary to store the information for the current row
        info_dict = {col: None for col in col_list}  # Initialize all columns with None

        info_dict.update({
            'ciq_id': row.get('ciq_id', ""),
            'person_id': row.get('person_id', ""),
            'CEO_Flag': row.get('CEO_Flag', ""),
            'ceo': row.get('Name', ""),
            'ceo_title': row.get('Position', ""),
            'start_year': row.get('Start_Year', ""),
            'end_year': row.get('End_Year', ""),
            'gender': row.get('Gender', ""),
            'age': row.get('Age', ""),
            'prev_title': row.get('Prev_Title', ""),
            'officer_start_yr': row.get('Officer_Start_Year', ""),
            'affiliations': "N/A"  # Since affiliations info is not available in the CSV data
        })

        # Append the info_dict to info_list
        info_list.append(info_dict)

    # Convert info_list to a DataFrame
    info_df = pd.DataFrame(info_list, columns=col_list)

    return info_df

# Get the extracted information DataFrame
info_df = extract_info(data_merged)

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

info_df.head(25)
info_df.describe()

#DF with all people, with CEO flag
#info_df.to_csv('people.csv', index=False)
#pickle.dump(info_df, open("info_df.p", "wb"))

# Keep only CEOs
ceo_final = info_df[info_df['CEO_Flag'] != 0]

# Sort ceo_final based on person_id and start_year in descending order
ceo_final = ceo_final.sort_values(by=['person_id', 'start_year'], ascending=[True, False])

# Drop duplicate entries based on person_id, keeping the first occurrence (later start date)
ceo_final = ceo_final.drop_duplicates(subset='person_id', keep='first')

# Drop these indices from the DataFrame
indices_to_drop = ceo_final.loc[ceo_final['end_year'].notnull()].index
ceo_current_final = ceo_final.drop(indices_to_drop)
#ceo_current_final.to_csv('ceos_current_final.csv', index=False)
#pickle.dump(ceo_current_final, open("ceos_current_final.p", "wb"))

# Reset the index of ceo_final (optional)
ceo_final.reset_index(drop=True, inplace=True)

# Output the final DataFrame to CSV and pickle files
ceo_final.to_csv('ceo_final.csv', index=False)
pickle.dump(ceo_final, open("ceo_final.p", "wb"))

###TO DO - create unique identifiers here and then use the CEO map on this

In [ ]:
# Step 1: Create a New DataFrame for Tracking Duplicate Entries
ceo_dup = ceo_final.copy()

# Sort the DataFrame and Reset the Index
ceo_dup.sort_values(['company','start_year','start_mo'], ascending=False, inplace=True)
ceo_dup.reset_index(drop=True, inplace=True)

# Initialize a new column to track duplicate entries
ceo_dup['dup_bool'] = False  # False indicates not a duplicate

# Step 2: Identify Duplicate Entries
for index, row in ceo_dup[:-1].iterrows():  # last row will throw out of range
    if ceo_dup.loc[index].permid == ceo_dup.loc[index+1].permid and ceo_dup.loc[index].person_id == ceo_dup.loc[index+1].person_id:
        ceo_dup.loc[index+1,'end_year'] = ceo_dup.loc[index,'end_year']
        ceo_dup.loc[index+1,'end_mo'] = ceo_dup.loc[index,'end_mo']
        ceo_dup.loc[index, 'dup_bool'] = True  # Mark this row as a duplicate

# Step 3: Filter Out Duplicate Entries
ceo_dedup = ceo_dup[ceo_dup['dup_bool'] != True].drop('dup_bool', axis=1)

# Output the final DataFrame to CSV and pickle files
#ceo_dedup.to_csv('ceo_dedup.csv', index=False)
pickle.dump(ceo_dedup, open("ceo_dedup.p", "wb"))

# Ensure 'end_year' is in numeric format, non-numeric values will be converted to NaN
ceo_dedup['end_year'] = pd.to_numeric(ceo_dedup['end_year'], errors='coerce')

# Filter out rows where 'end_year' is 2012 or earlier, but keep rows where 'end_year' is NaN
ceo_final_v2 = ceo_dedup.loc[(ceo_dedup['end_year'] >= 2008) | (ceo_dedup['end_year'].isna())]

# Group by 'ciq_id' and filter out groups with 3 or more occurrences
ceo_final_v2 = ceo_final_v2.groupby('ciq_id').filter(lambda x: len(x) < 3)

# Now, filtered_df will contain only the rows where 'ciq_id' is repeated less than 3 times

# Output the newly filtered data to a new DataFrame
ceo_final_v2.to_csv('ceo_final_v2.csv', index=False)
pickle.dump(ceo_final_v2, open("ceo_final_v2.p", "wb"))


In [ ]:
ciq_df = pd.read_csv('ciq_data_raw_20231127.csv', encoding='latin-1')

ciq_df.sort_values(['ciq_id','start_year'], inplace=True)

# Bring in historical values:
ciq_df['sp_adj_7'] = ciq_df.groupby(['ciq_id'])['sp_adj'].transform(lambda x: x.shift(7))
ciq_df['sp_adj_2'] = ciq_df.groupby(['ciq_id'])['sp_adj'].transform(lambda x: x.shift(2))
ciq_df['sp_adj_1x'] = ciq_df.groupby(['ciq_id'])['sp_adj'].transform(lambda x: x.shift(-1))
ciq_df['sp_adj_5x'] = ciq_df.groupby(['ciq_id'])['sp_adj'].transform(lambda x: x.shift(-5))

ciq_df['sales_7'] = ciq_df.groupby(['ciq_id'])['sales'].transform(lambda x: x.shift(7))
ciq_df['sales_2'] = ciq_df.groupby(['ciq_id'])['sales'].transform(lambda x: x.shift(2))
ciq_df['sales_1x'] = ciq_df.groupby(['ciq_id'])['sales'].transform(lambda x: x.shift(-1))
ciq_df['sales_5x'] = ciq_df.groupby(['ciq_id'])['sales'].transform(lambda x: x.shift(-5))

# Convert the necessary columns to numeric data type
cols_to_convert = ['sp_adj', 'sp_adj_7', 'sp_adj_2', 'sp_adj_1x', 'sp_adj_5x', 'sales', 'sales_7', 'sales_2', 'sales_1x', 'sales_5x']
for col in cols_to_convert:
    ciq_df[col] = pd.to_numeric(ciq_df[col], errors='coerce')

# Optionally fill NaN values with a default value (e.g., 0)
#ciq_df.fillna(0, inplace=True)

# Calculate growth rates:
ciq_df['tsr_5p'] = np.power((ciq_df['sp_adj_2']/ciq_df['sp_adj_7']), .2)-1
ciq_df['tsr_2'] = np.power((ciq_df['sp_adj']/ciq_df['sp_adj_2']), .5)-1
ciq_df['tsr_1x'] = np.power((ciq_df['sp_adj_1x']/ciq_df['sp_adj']), 1)-1
ciq_df['tsr_5x'] = np.power((ciq_df['sp_adj_5x']/ciq_df['sp_adj']), .2)-1

ciq_df['growth_5p'] = np.power((ciq_df['sales_2']/ciq_df['sales_7']), .2)-1
ciq_df['growth_2'] = np.power((ciq_df['sales']/ciq_df['sales_2']), .5)-1
ciq_df['growth_1x'] = np.power((ciq_df['sales_1x']/ciq_df['sales']), 1)-1
ciq_df['growth_5x'] = np.power((ciq_df['sales_5x']/ciq_df['sales']), .2)-1

ciq_df.head(25)

# Add variables for analyzing turnaround components
ciq_df['ebitda_5p'] = ciq_df.groupby(['ciq_id'])['ebitda'].transform(lambda x: x.shift(7))
ciq_df['ebitda_2'] = ciq_df.groupby(['ciq_id'])['ebitda'].transform(lambda x: x.shift(2))
ciq_df['ebitda_1x'] = ciq_df.groupby(['ciq_id'])['ebitda'].transform(lambda x: x.shift(-1))
ciq_df['ebitda_5x'] = ciq_df.groupby(['ciq_id'])['ebitda'].transform(lambda x: x.shift(-5))

ciq_df['eps_5p'] = ciq_df.groupby(['ciq_id'])['eps'].transform(lambda x: x.shift(7))
ciq_df['eps_2'] = ciq_df.groupby(['ciq_id'])['eps'].transform(lambda x: x.shift(2))
ciq_df['eps_1x'] = ciq_df.groupby(['ciq_id'])['eps'].transform(lambda x: x.shift(-1))
ciq_df['eps_5x'] = ciq_df.groupby(['ciq_id'])['eps'].transform(lambda x: x.shift(-5))

ciq_df['sp_5p'] = ciq_df.groupby(['ciq_id'])['sp'].transform(lambda x: x.shift(7))
ciq_df['sp_2'] = ciq_df.groupby(['ciq_id'])['sp'].transform(lambda x: x.shift(2))
ciq_df['sp_1x'] = ciq_df.groupby(['ciq_id'])['sp'].transform(lambda x: x.shift(-1))
ciq_df['sp_5x'] = ciq_df.groupby(['ciq_id'])['sp'].transform(lambda x: x.shift(-5))


ciq_df['sales_2x'] = ciq_df.groupby(['ciq_id'])['sales'].transform(lambda x: x.shift(-2))
ciq_df['capex_1x'] = ciq_df.groupby(['ciq_id'])['capex'].transform(lambda x: x.shift(-1))
ciq_df['capex_2x'] = ciq_df.groupby(['ciq_id'])['capex'].transform(lambda x: x.shift(-2))
ciq_df['r&d_1x'] = ciq_df.groupby(['ciq_id'])['r&d'].transform(lambda x: x.shift(-1))
ciq_df['r&d_2x'] = ciq_df.groupby(['ciq_id'])['r&d'].transform(lambda x: x.shift(-2))

# Convert the necessary columns to numeric data type
cols_to_convert = ['sp_adj', 'sp_adj_7', 'sp_adj_2', 'sp_adj_1x', 'sp_adj_5x',
                   'sales', 'sales_7', 'sales_2', 'sales_1x', 'sales_5x',
                   'ebitda', 'ebitda_5p', 'ebitda_2', 'ebitda_1x', 'ebitda_5x',
                   'eps', 'eps_5p', 'eps_2', 'eps_1x', 'eps_5x',
                   'sp', 'sp_5p', 'sp_2', 'sp_1x', 'sp_5x',
                   'capex', 'capex_1x', 'capex_2x',
                   'r&d', 'r&d_1x', 'r&d_2x']
for col in cols_to_convert:
    ciq_df[col] = pd.to_numeric(ciq_df[col], errors='coerce')

# Optionally fill NaN values with a default value (e.g., 0)
#ciq_df.fillna(0, inplace=True)

ciq_df['capex_sales'] = -1*ciq_df['capex']/ciq_df['sales']
ciq_df['capex_sales_1x'] = -1*ciq_df['capex_1x']/ciq_df['sales_1x']
ciq_df['capex_sales_2x'] = -1*ciq_df['capex_2x']/ciq_df['sales_2x']
ciq_df['capex_sales_avg'] = (ciq_df['capex_sales'] + ciq_df['capex_sales_1x'] + ciq_df['capex_sales_2x'])/3.0

ciq_df['r&d_sales'] = ciq_df['r&d']/ciq_df['sales']
ciq_df['r&d_sales_1x'] = ciq_df['r&d_1x']/ciq_df['sales_1x']
ciq_df['r&d_sales_2x'] = ciq_df['r&d_2x']/ciq_df['sales_2x']
ciq_df['r&d_sales_avg'] = (ciq_df['r&d_sales'] + ciq_df['r&d_sales_1x'] + ciq_df['r&d_sales_2x'])/3.0

ciq_df['restructuring_1x'] = ciq_df.groupby(['ciq_id'])['restructuring'].transform(lambda x: x.shift(-1))
ciq_df['tf_1x'] = ciq_df.groupby(['ciq_id'])['tf'].transform(lambda x: x.shift(-1))
#ciq_df.rename(columns={'restructuring':'restructuring_1x'}, inplace=True)
ciq_df.head(20)


# Replace infinite values with NaN
ciq_df.replace(np.inf, np.NaN, inplace=True)
#ciq_df.head()
ciq_df.to_csv('ciq_data_20231127.csv', index=False)
pickle.dump(ciq_df, open("ciq_data_20231127.p", "wb" ))

# Utilizing the describe function to provide a statistical summary of the DataFrame, which can be useful for analysis.
ciq_df.describe()

In [ ]:
import pandas as pd
import numpy as np
import pickle
import re
#!pip install pycountry
#import pycountry

# Load CEO data
ceo_final = pickle.load(open("ceo_final_v2.p", "rb" ))

# Load CIQ data
ciq_df = pickle.load(open("ciq_data_20231127.p", "rb" ))

# Load industry sector data
helper_2 = pd.read_csv('231120_company_id.csv')[['ciq_id','industry_sector']]
helper_2.rename(columns={'industry_sector':'sector'}, inplace=True)

# Compute 'start_year_int' by converting 'start_year' to numeric, converting errors to NaN, and then filling NaN with 1000
ceo_final['start_year_int'] = ceo_final['start_year'].apply(lambda x: pd.to_numeric(x, errors='ignore')).fillna(1000)

# Same conversion for 'end_year_int'
ceo_final['end_year_int'] = ceo_final['end_year'].apply(lambda x: pd.to_numeric(x, errors='ignore')).fillna(3000)

# Identify interim CEOs where 'end_year_int' is equal to 'start_year_int'
ceo_final['interim'] = np.where((ceo_final['end_year_int'] == ceo_final['start_year_int']), 1, 0)

duplicate_count = ceo_final.duplicated(subset='ciq_id', keep=False).sum()
print(f'Count of duplicate entries based on ciq_id: {duplicate_count}')

ceo_final.describe()

# Filter out interim CEOs
ceo_final = ceo_final[ceo_final['interim'] == 0].copy()  # Use copy to avoid SettingWithCopyWarning

# Drop unnecessary columns
ceo_final.drop(['start_year_int', 'end_year_int', 'interim'], axis=1, inplace=True)

ceo_final.describe()

#ceo_final_premerge = ceo_final.to_csv('ceo_final_premerge.csv', index=False)


# First Merge
merged_df = ceo_final.merge(ciq_df, on=['ciq_id', 'start_year'], how='right')
# Before the first merge
print("\After first merge:")
print(merged_df['start_year'].value_counts())

#ceo_final_merge = merged_df.to_csv('ceo_final_merge.csv', index=False)

# Second Merge
merged_df = merged_df.merge(helper_2, on='ciq_id', how='right')
print("\nAfter third merge:")
print(merged_df['start_year'].value_counts())

#ceo_final_merge2 = merged_df.to_csv('ceo_final_merge2.csv', index=False)

# Save Output
ceo_final = merged_df
ceo_final.to_csv('ceo_final.csv', index=False)
pickle.dump(merged_df, open("ceo_final.p", "wb" ))

In [ ]:
# Load industry data from a CSV file into a DataFrame
ind_df = pd.read_csv("industry_data_20231127.csv")
# Extract the sector information from the 'industry' column and create a new column 'sector'
ind_df['sector'] = ind_df['industry'].map(lambda x: x[8:-9])
# Rename the sector 'Health Care' to 'Healthcare' for consistency
#ind_df.loc[ind_df.sector=="Health Care", 'sector'] = "Healthcare"
ind_df.loc[ind_df.sector=="Telecommunication Services", 'sector'] = "Communication Services"

# Drop unwanted columns from the 'ind_df' DataFrame
ind_df.drop(['industry', 'index', 'date', 'ciq total return gross', 'check industry', 'sp', 'growth'], axis=1, inplace=True)
# Rename the remaining columns for clarity and consistency
ind_df.rename(columns={'total return':'ind_sp', 'year':'start_year', 'ciq':'ciq_id'}, inplace=True)
# Filter out rows where 'ind_sp' is 0 as these are likely to be invalid entries
ind_df = ind_df[ind_df.ind_sp != 0]

# Calculate the historical total shareholder return (TSR) values for each industry
# by shifting the 'ind_sp' values by different time periods (7, 2, -1, and -5 years)
ind_df['ind_sp_5p'] = ind_df.groupby(['ciq_id'])['ind_sp'].transform(lambda x: x.shift(7))
ind_df['ind_sp_2'] = ind_df.groupby(['ciq_id'])['ind_sp'].transform(lambda x: x.shift(2))
ind_df['ind_sp_1x'] = ind_df.groupby(['ciq_id'])['ind_sp'].transform(lambda x: x.shift(-1))
ind_df['ind_sp_5x'] = ind_df.groupby(['ciq_id'])['ind_sp'].transform(lambda x: x.shift(-5))

# Calculate the TSR growth rates for each industry over different time periods (5-year, 2-year, 1-year forward, and 5-year forward)
ind_df['ind_tsr_5p'] = np.power((ind_df['ind_sp_2']/ind_df['ind_sp_5p']), .2)-1
ind_df['ind_tsr_2'] = np.power((ind_df['ind_sp']/ind_df['ind_sp_2']), .5)-1
ind_df['ind_tsr_1x'] = np.power((ind_df['ind_sp_1x']/ind_df['ind_sp']), 1)-1
ind_df['ind_tsr_5x'] = np.power((ind_df['ind_sp_5x']/ind_df['ind_sp']), .2)-1

# Display the first 20 rows of the 'ind_df' DataFrame to verify the calculations
ind_df.head(20)

# Merge the industry TSR growth rates with the 'ceo_full' DataFrame on the 'start_year' and 'sector' columns
ceo_final = ceo_final.merge(ind_df[['start_year', 'sector', 'ind_tsr_2', 'ind_tsr_1x', 'ind_tsr_5x', 'ind_tsr_5p']], on=['start_year', 'sector'], how='left')
# Uncomment the following line to display the first 5 rows of the 'ceo_full' DataFrame to verify the merge
#ceo_full.head()
# Uncomment the following line to display the column names of the 'ind_df' DataFrame
#ind_df.columns

# Calculate the difference between the company TSR and industry TSR for different time periods
ceo_final['tsr_5p_delta'] = ceo_final['tsr_5p'] - ceo_final['ind_tsr_5p']
ceo_final['tsr_2_delta'] = ceo_final['tsr_2'] - ceo_final['ind_tsr_2']
ceo_final['tsr_1x_delta'] = ceo_final['tsr_1x'] - ceo_final['ind_tsr_1x']
ceo_final['tsr_5x_delta'] = ceo_final['tsr_5x'] - ceo_final['ind_tsr_5x']

# Display the first 20 rows of the 'ceo_full' DataFrame to verify the calculations
ceo_final.head(20)

# Calculate the change in TSR growth (vs prior period) for different time periods
ceo_final['tsr_2_chg'] = ceo_final['tsr_2_delta'] - ceo_final['tsr_5p_delta']
ceo_final['tsr_1x_chg'] = ceo_final['tsr_1x_delta'] - ceo_final['tsr_2_delta']
ceo_final['tsr_5x_chg'] = ceo_final['tsr_5x_delta'] - ceo_final['tsr_2_delta']

# Create bins and labels for categorizing the change in TSR growth
# The bins are defined by the range of TSR growth rates, and the labels represent different categories of growth
bins = [-999]
lbls = ["--"]
for i in range(-50,55,5):
    bins.append(float(i)/100.0)
    lbls.append(str(i))
lbls.remove("0")
bins.append(999)
lbls.append("++")

# Create new columns to categorize the change in TSR growth for different time periods
ceo_final['2_chg_lbl'] = pd.cut(ceo_final['tsr_2_chg'], bins, labels=lbls)
ceo_final['1x_chg_lbl'] = pd.cut(ceo_final['tsr_1x_chg'], bins, labels=lbls)
ceo_final['5x_chg_lbl'] = pd.cut(ceo_final['tsr_5x_chg'], bins, labels=lbls)

# Create big bins for color-coding the change in TSR growth
bins_2 = [-999,-.1,0,.1,999]
lbls_2 = ["V_neg","neg","pos","V_pos"]

# Create new columns to categorize the change in TSR growth using big bins for color-coding
ceo_final['cat_2'] = pd.cut(ceo_final['tsr_2_chg'], bins_2, labels=lbls_2)
ceo_final['cat_1x'] = pd.cut(ceo_final['tsr_1x_chg'], bins_2, labels=lbls_2)
ceo_final['cat_5x'] = pd.cut(ceo_final['tsr_5x_chg'], bins_2, labels=lbls_2)

ceo_final.describe()

In [ ]:
# Create new columns to calculate the growth rates of EBITDA, EPS, and SP for different time periods
ceo_final['ebitda_2_growth'] = np.power((ceo_final['ebitda']/ceo_final['ebitda_2']), .5) - 1
ceo_final['eps_2_growth'] = np.power((ceo_final['eps']/ceo_final['eps_2']), .5) - 1
ceo_final['sp_2_growth'] = np.power((ceo_final['sp']/ceo_final['sp_2']), .5) - 1

ceo_final['ebitda_1x_growth'] = ceo_final['ebitda_1x']/ceo_final['ebitda'] - 1
ceo_final['eps_1x_growth'] = ceo_final['eps_1x']/ceo_final['eps'] - 1
ceo_final['sp_1x_growth'] = ceo_final['sp_1x']/ceo_final['sp'] - 1

ceo_final['ebitda_5x_growth'] = np.power((ceo_final['ebitda_5x']/ceo_final['ebitda']), .2) - 1
ceo_final['eps_5x_growth'] = np.power((ceo_final['eps_5x']/ceo_final['eps']), .2) - 1
ceo_final['sp_5x_growth'] = np.power((ceo_final['sp_5x']/ceo_final['sp']), .2) - 1

# Calculate the contribution of top-line sales growth, cost control (earnings growth),
# cash management (cash growth), and market expectations (price growth) to TSR
# The calculations are done for different time periods (2-year, 1-year forward, and 5-year forward)
# to analyze the turnaround components

# * 2 *
# For the 2-year period
ceo_final['sales_2_contrib'] = ceo_final['growth_2']
ceo_final['earnings_2_contrib'] = ceo_final['eps_2_growth'] - ceo_final['sales_2_contrib']
ceo_final['cash_2_contrib'] = ceo_final['tsr_2'] - ceo_final['sp_2_growth']

# Corrections:
# If eps is blank, use ebitda
ceo_final['earnings_2_contrib'] = np.where((np.isnan(ceo_final['earnings_2_contrib'])), ceo_final['ebitda_2_growth'] - ceo_final['sales_2_contrib'], ceo_final['earnings_2_contrib'])

# If earnings go from negative to positive, set contribution = total TSR growth (vice versa):
ceo_final['earnings_2_contrib'] = np.where(((ceo_final['eps_2'] < 0) & (ceo_final['eps'] > 0)), ceo_final['tsr_2'], ceo_final['earnings_2_contrib'])
ceo_final['earnings_2_contrib'] = np.where(((ceo_final['eps_2'] > 0) & (ceo_final['eps'] < 0)), ceo_final['tsr_2'], ceo_final['earnings_2_contrib'])

# If earnings are still negative, invert growth rate:
ceo_final['earnings_2_contrib'] = np.where((ceo_final['eps'] < 0), -1 * ceo_final['earnings_2_contrib'], ceo_final['earnings_2_contrib'])

# Cap earnings contribution at 100% - arbitrary but need to cut off the 88x growth contribution
ceo_final['earnings_2_contrib'] = np.where((ceo_final['earnings_2_contrib'] < -1), -1, ceo_final['earnings_2_contrib'])
ceo_final['earnings_2_contrib'] = np.where((ceo_final['earnings_2_contrib'] > 1), 1, ceo_final['earnings_2_contrib'])

ceo_final['price_2_contrib'] = ceo_final['tsr_2'] - ceo_final['sales_2_contrib'] - ceo_final['earnings_2_contrib'] - ceo_final['cash_2_contrib']

# * 1x *

# Decomposing TSR for a 1-year period
ceo_final['sales_1x_contrib'] = ceo_final['growth_1x']
ceo_final['earnings_1x_contrib'] = ceo_final['eps_1x_growth'] - ceo_final['sales_1x_contrib']
ceo_final['cash_1x_contrib'] = ceo_final['tsr_1x'] - ceo_final['sp_1x_growth']

# Corrections:
# If eps is blank, use ebitda
ceo_final['earnings_1x_contrib'] = np.where((np.isnan(ceo_final['earnings_1x_contrib'])), ceo_final['ebitda_1x_growth'] - ceo_final['sales_1x_contrib'], ceo_final['earnings_1x_contrib'])

# If earnings go from negative to positive, set contribution = total TSR growth:
ceo_final['earnings_1x_contrib'] = np.where(((ceo_final['eps'] < 0) & (ceo_final['eps_1x'] > 0)), ceo_final['tsr_1x'], ceo_final['earnings_1x_contrib'])
ceo_final['earnings_1x_contrib'] = np.where(((ceo_final['eps'] > 0) & (ceo_final['eps_1x'] < 0)), ceo_final['tsr_1x'], ceo_final['earnings_1x_contrib'])

# If earnings are still negative, invert growth rate:
ceo_final['earnings_1x_contrib'] = np.where((ceo_final['eps_1x'] < 0), -1 * ceo_final['earnings_1x_contrib'], ceo_final['earnings_1x_contrib'])

# Cap earnings contribution at 100% - arbitrary but need to cut off the 88x growth contribution
ceo_final['earnings_1x_contrib'] = np.where((ceo_final['earnings_1x_contrib'] < -1), -1, ceo_final['earnings_1x_contrib'])
ceo_final['earnings_1x_contrib'] = np.where((ceo_final['earnings_1x_contrib'] > 1), 1, ceo_final['earnings_1x_contrib'])

ceo_final['price_1x_contrib'] = ceo_final['tsr_1x'] - ceo_final['sales_1x_contrib'] - ceo_final['earnings_1x_contrib'] - ceo_final['cash_1x_contrib']

# * 5x *

# Decomposing TSR for a 5-year period
ceo_final['sales_5x_contrib'] = ceo_final['growth_5x']
ceo_final['earnings_5x_contrib'] = ceo_final['eps_5x_growth'] - ceo_final['sales_5x_contrib']
ceo_final['cash_5x_contrib'] = ceo_final['tsr_5x'] - ceo_final['sp_5x_growth']

# Corrections:
# If eps is blank, use ebitda
ceo_final['earnings_5x_contrib'] = np.where((np.isnan(ceo_final['earnings_5x_contrib'])), ceo_final['ebitda_5x_growth'] - ceo_final['sales_5x_contrib'], ceo_final['earnings_5x_contrib'])

# If earnings go from negative to positive, set contribution = total TSR growth:
ceo_final['earnings_5x_contrib'] = np.where(((ceo_final['eps'] < 0) & (ceo_final['eps_5x'] > 0)), ceo_final['tsr_5x'], ceo_final['earnings_5x_contrib'])
ceo_final['earnings_5x_contrib'] = np.where(((ceo_final['eps'] > 0) & (ceo_final['eps_5x'] < 0)), ceo_final['tsr_5x'], ceo_final['earnings_5x_contrib'])

# If earnings are still negative, invert growth rate:
ceo_final['earnings_5x_contrib'] = np.where((ceo_final['eps_5x'] < 0), -1 * ceo_final['earnings_5x_contrib'], ceo_final['earnings_5x_contrib'])

# Cap earnings contribution at 100% - arbitrary but need to cut off the 88x growth contribution
ceo_final['earnings_5x_contrib'] = np.where((ceo_final['earnings_5x_contrib'] < -1), -1, ceo_final['earnings_5x_contrib'])
ceo_final['earnings_5x_contrib'] = np.where((ceo_final['earnings_5x_contrib'] > 1), 1, ceo_final['earnings_5x_contrib'])

ceo_final['price_5x_contrib'] = ceo_final['tsr_5x'] - ceo_final['sales_5x_contrib'] - ceo_final['earnings_5x_contrib'] - ceo_final['cash_5x_contrib']

# Utilizing the describe function to provide a statistical summary of the DataFrame, which can be useful for analysis.
ceo_final.describe()

In [ ]:
# Step 1: Load the data
df_nlp = pd.read_csv("231127_nlp.csv")

# Step 2: Impute 2022 Data
df_nlp_2022 = df_nlp[df_nlp['year'] == 2021].copy()
df_nlp_2022['year'] = 2022
data = pd.concat([df_nlp, df_nlp_2022], axis=0)

# Step 3: Impute for Companies with No Values
industry_avg = data.groupby(['industry', 'year']).mean().reset_index()

def fill_with_industry_avg(row):
    if pd.isnull(row['biological']):
        row['biological'] = industry_avg[(industry_avg['industry'] == row['industry']) & (industry_avg['year'] == row['year'])]['biological'].values[0]
    if pd.isnull(row['longterm']):
        row['longterm'] = industry_avg[(industry_avg['industry'] == row['industry']) & (industry_avg['year'] == row['year'])]['longterm'].values[0]
    return row

df_nlp = data.apply(fill_with_industry_avg, axis=1)

# Step 4: Impute Values for Companies with Partial Data
def fill_with_company_avg(group):
    avg_biological = group['biological'].mean()
    avg_longterm = group['longterm'].mean()
    group['biological'].fillna(avg_biological, inplace=True)
    group['longterm'].fillna(avg_longterm, inplace=True)
    return group

df_nlp = df_nlp.groupby('name').apply(fill_with_company_avg)

# Step 5: Save the Data
df_nlp.to_csv("231127_nlp_clean.csv", index=False)
pickle.dump(df_nlp, open("231121_nlp_clean.p", "wb" ))

# Filter the data to include only the MD&A section (section 56), and then calculate the mean NLP scores per company per year.
df_nlp = df_nlp.groupby(["ciq_id", "year"])[['biological', 'longterm', 'purpose']].mean().reset_index()

# Adjust the year to account for the fact that 10-Ks are published in the year following the year they report on.
df_nlp["year"] = df_nlp["year"] - 1

# Calculate net scores for 'longterm' and 'biological' by subtracting other relevant scores. This provides a measure of the net orientation of each firm's strategy.
df_nlp["net_longterm"] = df_nlp["longterm"] - df_nlp["biological"]

# Standardize the net scores by year to have a mean of 0 and a standard deviation of 1, making them comparable across years.
nlp_scores = ['net_longterm']
df_nlp_std = df_nlp.copy()
for var in nlp_scores:
    df_nlp_std[var] = df_nlp_std.groupby("year")[var].transform(lambda x: (x - x.mean()) / x.std())

# Select only the columns needed for further analysis.
df_nlp_std = df_nlp_std[['ciq_id','year','net_longterm']]

# Display summary statistics of the standardized net scores to understand their distribution.
df_nlp_std.describe()

# Rename the 'year' column to 'start_year' to match the naming convention in other DataFrames.
df_nlp_std.rename(columns={'year':'start_year'}, inplace=True)

# Merge the standardized net scores into the main DataFrame 'ceo_final_v2'.
ceo_final_v2 = ceo_final.merge(df_nlp_std, how='left', on=['ciq_id','start_year']).sort_values(['ciq_id','start_year'])
#ceo_final_v2.head(20)

# Shift the standardized net scores by one year forward and two years forward to create lagged variables.
# This is done to analyze the effect of past BHI strategy scores on later financial performance.
ceo_final_v2['net_longterm_1x'] = ceo_final_v2.groupby(['ciq_id'])['net_longterm'].transform(lambda x: x.shift(-1))
ceo_final_v2['net_longterm_2x'] = ceo_final_v2.groupby(['ciq_id'])['net_longterm'].transform(lambda x: x.shift(-2))

# Calculate the 3-year average of the standardized net longterm scores.
ceo_final_v2['net_longterm_avg'] = (ceo_final_v2['net_longterm'] + ceo_final_v2['net_longterm_1x'] + ceo_final_v2['net_longterm_2x']) / 3.0

# Similar operations are performed to create lagged variables for the CEO name and title.
ceo_final_v2['ceo_1x'] = ceo_final_v2.groupby(['ciq_id'])['ceo'].transform(lambda x: x.shift(-1))
ceo_final_v2['ceo_title_1x'] = ceo_final_v2.groupby(['ciq_id'])['prev_title'].transform(lambda x: x.shift(-1))
####UNCLEAR HOW THIS WAS DONE BEFORE; MAKING ASSUMPTION
ceo_final_v2['name_x'] = ceo_final_v2.groupby(['ciq_id'])['ceo'].transform(lambda x: x.shift(-1))

ceo_final_v2 = ceo_final_v2
ceo_final_v2.to_csv('ceo_final_v2.csv', index=False)

In [ ]:
# save old version as df_all
df_all = ceo_final_v2
df_all.to_csv('df_all.csv', index=False)

# Remove if max market cap < $5B
#ceo_final_v2['max_mc'] = ceo_final_v2.groupby('ciq_id')['mc'].transform(lambda x: x.max())
#ceo_final_v2 = ceo_final_v2[ceo_final_v2['max_mc']>=5000]
#ceo_final_v2.drop('max_mc',axis=1, inplace=True)

#ceo_final_v2.describe()


# merge in company name
helper_df = pd.read_csv('231120_company_id.csv')[['ciq_id','name']]

ceo_final_v2 = ceo_final_v2.merge(helper_df, how='left', on='ciq_id')
ceo_final_v2.describe(include='all')

'''
# # TEMPORARY
# exclude mutual funds, etc.
fund_list = pd.read_csv('fund_list.csv', index_col=False)[['ciq','fund']]
fund_list.rename(columns={'ciq': 'ciq_id'}, inplace=True)
ceo_temp = ceo_final_v2.merge(fund_list, how='left', on='ciq_id')

ceo_temp = ceo_temp[ceo_temp['fund']!='Y'].drop('fund',axis=1)
ceo_temp.describe()'''

# export
#ceo_temp.to_csv('output_20170814_all.csv', index=False)
ceo_final_v2.to_csv('output_20231127.csv', index=False)
pickle.dump(ceo_final, open("output_20231127.p", "wb" ))

In [ ]:
# Generate descriptive statistics for the DataFrame to understand the overall distribution of the data.
df_all.describe(include='all')

# Convert columns to numeric data type, if necessary
df_all[['mc','capex', 'sales', 'r&d']] = df_all[['mc','capex', 'sales', 'r&d']].apply(pd.to_numeric, errors='coerce')
# Replace non-numeric values with NaN or 0, as appropriate
df_all[['mc','capex', 'sales', 'r&d']] = df_all[['mc','capex', 'sales', 'r&d']].replace('--', np.nan).fillna(0)


# Calculate the total earnings and cash-adjusted market cap for each company.
# Calculating shares outstanding by dividing market cap by stock price.
df_all['shares'] = df_all['mc'] / df_all['sp']
# Calculating total earnings by multiplying earnings per share by shares outstanding.
df_all['earnings_tot'] = df_all['eps'] * df_all['shares']
# Setting negative earnings to zero as they are considered non-meaningful for this analysis.
df_all['earnings_tot'] = np.where(df_all['earnings_tot'] < 0, 0, df_all['earnings_tot'])
# Calculating cash-adjusted market cap by multiplying adjusted stock price by shares outstanding.
df_all['mc_adj'] = df_all['sp_adj'] * df_all['shares']
# Filling any missing values with zero in the 'shares', 'earnings_tot', and 'mc_adj' columns.
df_all[['shares','earnings_tot','mc_adj']] = df_all[['shares','earnings_tot','mc_adj']].fillna(value=0)

# Calculate industry aggregates to understand the overall trends in the data.
# Summing up capital expenditures, sales, earnings, market cap, and cash-adjusted market cap for each sector in each year.
agg_df = df_all[df_all['sp']>0].groupby(["sector", "start_year"])[['capex','sales','earnings_tot','mc','mc_adj']].sum().reset_index()
# Merging the industry TSR data into the industry aggregate DataFrame.
agg_df = agg_df.merge(df_all.groupby(['sector','start_year'])[['ind_tsr_5p','ind_tsr_2','ind_tsr_1x','ind_tsr_5x']].first().reset_index(), how='left',on=['sector','start_year'])

agg_df.head(10)

# Create lagged variables for the industry-adjusted market cap to analyze the growth over different periods.
agg_df['ind_mc_adj_5p'] = agg_df.groupby(['sector'])['mc_adj'].transform(lambda x: x.shift(7))
agg_df['ind_mc_adj_2'] = agg_df.groupby(['sector'])['mc_adj'].transform(lambda x: x.shift(2))
agg_df['ind_mc_adj_1x'] = agg_df.groupby(['sector'])['mc_adj'].transform(lambda x: x.shift(-1))
agg_df['ind_mc_adj_5x'] = agg_df.groupby(['sector'])['mc_adj'].transform(lambda x: x.shift(-5))

# Rename the column for better readability.
agg_df.rename(columns={'mc_adj':'ind_mc_adj'}, inplace=True)

# Calculate implied industry TSR (Total Shareholder Return) based on the growth of the industry-adjusted market cap.
agg_df['ind_tsr_implied_5p'] = np.power((agg_df['ind_mc_adj_2']/agg_df['ind_mc_adj_5p']), .2)-1
agg_df['ind_tsr_implied_2'] = np.power((agg_df['ind_mc_adj']/agg_df['ind_mc_adj_2']), .5)-1
agg_df['ind_tsr_implied_1x'] = np.power((agg_df['ind_mc_adj_1x']/agg_df['ind_mc_adj']), 1)-1
agg_df['ind_tsr_implied_5x'] = np.power((agg_df['ind_mc_adj_5x']/agg_df['ind_mc_adj']), .2)-1

# Calculate the change in industry TSR over different periods.
agg_df['ind_tsr_chg_2'] = agg_df['ind_tsr_implied_2'] - agg_df['ind_tsr_implied_5p']
agg_df['ind_tsr_chg_1x'] = agg_df['ind_tsr_implied_1x'] - agg_df['ind_tsr_implied_2']
agg_df['ind_tsr_chg_5x'] = agg_df['ind_tsr_implied_5x'] - agg_df['ind_tsr_implied_2']

# Create lagged variables for other financial metrics to analyze the growth over different periods.
for i in ['sales', 'earnings_tot', 'mc']:
    agg_df[i+'_5p'] = agg_df.groupby(['sector'])[i].transform(lambda x: x.shift(7))
    agg_df[i+'_2'] = agg_df.groupby(['sector'])[i].transform(lambda x: x.shift(2))
    agg_df[i+'_1x'] = agg_df.groupby(['sector'])[i].transform(lambda x: x.shift(-1))
    agg_df[i+'_5x'] = agg_df.groupby(['sector'])[i].transform(lambda x: x.shift(-5))

    # Calculate the growth rate of various financial metrics over different periods.
    agg_df[i+'_5p_growth'] = np.power((agg_df[i]/agg_df[i+'_5p']),.2) - 1
    agg_df[i+'_2_growth'] = np.power((agg_df[i]/agg_df[i+'_2']),.5) - 1
    agg_df[i+'_1x_growth'] = np.power((agg_df[i+'_1x']/agg_df[i]),1) - 1
    agg_df[i+'_5x_growth'] = np.power((agg_df[i+'_5x']/agg_df[i]),.2) - 1

agg_df.describe()

# Top-line contribution = sales growth %
# Cost contribution = (earnings growth % - sales contrib.)
# Cash contribution = (sp-adj growth - sp growth)
# Expectations contribution = everything else (= P/E change)

# Calculate the contributions of sales, earnings, cash, and price to the industry TSR for different periods.
agg_df['ind_sales_2_contrib'] = agg_df['sales_2_growth']
agg_df['ind_earnings_2_contrib'] = agg_df['earnings_tot_2_growth'] - agg_df['ind_sales_2_contrib']
agg_df['ind_earnings_2_contrib'] = np.where(agg_df['ind_earnings_2_contrib']>0.5, 0.5, agg_df['ind_earnings_2_contrib'])
agg_df['ind_earnings_2_contrib'] = np.where(agg_df['ind_earnings_2_contrib']<-0.5, -0.5, agg_df['ind_earnings_2_contrib'])
# Cap earnings contribution at 50% - arbitrary but needed to cut off extreme values.
agg_df['ind_cash_2_contrib'] = agg_df['ind_tsr_implied_2'] - agg_df['mc_2_growth']
agg_df['ind_price_2_contrib'] = agg_df['ind_tsr_implied_2'] - agg_df['ind_sales_2_contrib'] - agg_df['ind_earnings_2_contrib'] - agg_df['ind_cash_2_contrib']

for i in ['ind_sales_2_contrib','ind_earnings_2_contrib','ind_cash_2_contrib','ind_price_2_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_2']-agg_df['ind_tsr_implied_2'])/4.0

# Similar calculations are done for 1-year and 5-year periods.
agg_df['ind_sales_1x_contrib'] = agg_df['sales_1x_growth']
agg_df['ind_earnings_1x_contrib'] = agg_df['earnings_tot_1x_growth'] - agg_df['ind_sales_1x_contrib']
agg_df['ind_earnings_1x_contrib'] = np.where(agg_df['ind_earnings_1x_contrib']>0.5, 0.5, agg_df['ind_earnings_1x_contrib'])
agg_df['ind_earnings_1x_contrib'] = np.where(agg_df['ind_earnings_1x_contrib']<-0.5, -0.5, agg_df['ind_earnings_1x_contrib'])
# Cap earnings contribution at 50% - arbitrary but needed to cut off extreme values.
agg_df['ind_cash_1x_contrib'] = agg_df['ind_tsr_implied_1x'] - agg_df['mc_1x_growth']
agg_df['ind_price_1x_contrib'] = agg_df['ind_tsr_implied_1x'] - agg_df['ind_sales_1x_contrib'] - agg_df['ind_earnings_1x_contrib'] - agg_df['ind_cash_1x_contrib']

for i in ['ind_sales_1x_contrib','ind_earnings_1x_contrib','ind_cash_1x_contrib','ind_price_1x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_1x']-agg_df['ind_tsr_implied_1x'])/4.0

agg_df['ind_sales_5x_contrib'] = agg_df['sales_5x_growth']
agg_df['ind_earnings_5x_contrib'] = agg_df['earnings_tot_5x_growth'] - agg_df['ind_sales_5x_contrib']
agg_df['ind_earnings_5x_contrib'] = np.where(agg_df['ind_earnings_5x_contrib']>0.5, 0.5, agg_df['ind_earnings_5x_contrib'])
agg_df['ind_earnings_5x_contrib'] = np.where(agg_df['ind_earnings_5x_contrib']<-0.5, -0.5, agg_df['ind_earnings_5x_contrib'])
# Cap earnings contribution at 50% - arbitrary but needed to cut off extreme values.
agg_df['ind_cash_5x_contrib'] = agg_df['ind_tsr_implied_5x'] - agg_df['mc_5x_growth']
agg_df['ind_price_5x_contrib'] = agg_df['ind_tsr_implied_5x'] - agg_df['ind_sales_5x_contrib'] - agg_df['ind_earnings_5x_contrib'] - agg_df['ind_cash_5x_contrib']

for i in ['ind_sales_5x_contrib','ind_earnings_5x_contrib','ind_cash_5x_contrib','ind_price_5x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_5x']-agg_df['ind_tsr_implied_5x'])/4.0

#agg_df.to_csv('industry_aggregate_test_20170804.csv', index=False)

# Create a 3-year average capex-sales ratio for benchmarking purposes.
agg_df['capex_sales'] = -1*agg_df['capex'] / agg_df['sales']
agg_df['capex_sales_1'] = agg_df.groupby(['sector'])['capex_sales'].transform(lambda x: x.shift(-1))
agg_df['capex_sales_2'] = agg_df.groupby(['sector'])['capex_sales'].transform(lambda x: x.shift(-2))
agg_df['ind_capex_sales_avg'] = (agg_df['capex_sales'] + agg_df['capex_sales_1'] + agg_df['capex_sales_2']) / 3.0

# Create a 3-year average R&D-sales ratio for firms that report R&D.
rd_df = df_all[df_all['r&d']>0].groupby(["sector", "start_year"])[['sales','r&d']].sum().reset_index()
rd_df['r&d_sales'] = rd_df['r&d'] / rd_df['sales']
rd_df['r&d_sales_1'] = rd_df.groupby(['sector'])['r&d_sales'].transform(lambda x: x.shift(-1))
rd_df['r&d_sales_2'] = rd_df.groupby(['sector'])['r&d_sales'].transform(lambda x: x.shift(-2))
rd_df['ind_r&d_sales_avg'] = (rd_df['r&d_sales'] + rd_df['r&d_sales_1'] + rd_df['r&d_sales_2']) / 3.0

merge_df = agg_df[['sector','start_year','ind_sales_2_contrib','ind_earnings_2_contrib','ind_cash_2_contrib','ind_price_2_contrib','ind_sales_1x_contrib','ind_earnings_1x_contrib','ind_cash_1x_contrib','ind_price_1x_contrib','ind_sales_5x_contrib','ind_earnings_5x_contrib','ind_cash_5x_contrib','ind_price_5x_contrib','ind_capex_sales_avg']].merge(rd_df[['sector','start_year','ind_r&d_sales_avg']])

# Merge the industry aggregate data into the master DataFrame.
ceo_out = df_all.merge(merge_df, how='left', on=['sector','start_year'])
ceo_out.describe()

ceo_out.to_csv('ceo_out.csv', index = False)


"""### EXPORT TURNAROUND SAMPLE:"""

ceo_sample = ceo_out

# CRITERIA:

# Market cap should be greater than 10B.
#ceo_sample = ceo_sample[ceo_sample['mc'] > 10000]

print(ceo_sample['start_year'].value_counts())

# All TSRs should be known.
#ceo_sample.dropna(subset=['tsr_5p','tsr_5x','ind_tsr_5p','ind_tsr_5x'], how='any', inplace=True)
# All TSRs should be known.
ceo_sample.dropna(subset=['tsr_2','tsr_1x','ind_tsr_2','ind_tsr_1x'], how='any', inplace=True)

print(ceo_sample['start_year'].value_counts())

# Only include odd years (to avoid overlapping 2-year samples) and years after 2004.
ceo_sample = ceo_sample[ceo_sample['start_year'] % 2 == 0]
ceo_sample = ceo_sample[ceo_sample['start_year'] > 2007]

print(ceo_sample['start_year'].value_counts())

# Exclude mutual funds, etc.
#fund_list = pd.read_csv('fund_list.csv', index_col=False)[['ciq','fund']]
#fund_list.rename(columns={'ciq': 'ciq_id'}, inplace=True)
#ceo_sample = ceo_sample.merge(fund_list, how='left', on='ciq_id')

#ceo_sample = ceo_sample[ceo_sample['fund']!='Y'].drop('fund',axis=1)

#ceo_sample.describe()

# Create a new column 'ceo_bool' to indicate whether the CEO data is available or not.
ceo_sample['ceo_bool'] = np.where( ((pd.isnull(ceo_sample['ceo'])) & (pd.isnull(ceo_sample['ceo_1x']))),"N","Y")
ceo_sample.head()

print(ceo_sample['start_year'].value_counts())

# Export the sample to a CSV file for further analysis.
ceo_sample.to_csv('ceo_output_20231127_sample.csv', index=False)

df_decline = ceo_sample[ceo_sample['cat_2']=='V_neg']
#df_decline.describe()

df_decline.to_csv('ceo_output_20231127_decline.csv', index=False)
pickle.dump(df_decline, open('ceo_output_20231127_decline.p', 'wb'))

In [ ]:
import numpy as np
import pandas as pd

# Load your dataframe
# df_all = pd.read_csv("your_data.csv")

# Ensure the correct columns are of numeric type
cols_to_convert = ['sp', 'sp_adj', 'sales', 'ebitda', 'eps', 'capex', 'mc', 'r&d', 'restructuring']
for col in cols_to_convert:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# Generate descriptive statistics
df_all.describe(include='all')

# Handle divisions ensuring we don't divide by zero
df_all['shares'] = np.where(df_all['sp'] != 0, df_all['mc'] / df_all['sp'], 0)
df_all['earnings_tot'] = df_all['eps'] * df_all['shares']
df_all['earnings_tot'] = np.where(df_all['earnings_tot'] < 0, 0, df_all['earnings_tot'])
df_all['mc_adj'] = df_all['sp_adj'] * df_all['shares']

# Fill NaN values in relevant columns
df_all[['shares', 'earnings_tot', 'mc_adj']] = df_all[['shares', 'earnings_tot', 'mc_adj']].fillna(value=0)

# Calculate industry aggregates
agg_df = df_all[df_all['sp'] > 0].groupby(["sector", "start_year"])[['capex', 'sales', 'earnings_tot', 'mc', 'mc_adj']].sum().reset_index()
agg_df = agg_df.merge(df_all.groupby(['sector', 'start_year'])[['ind_tsr_5p', 'ind_tsr_2', 'ind_tsr_1x', 'ind_tsr_5x']].first().reset_index(), how='left', on=['sector', 'start_year'])

# Handle divisions ensuring we don't divide by zero for capex-sales ratio
agg_df['capex_sales'] = np.where(agg_df['sales'] != 0, -1 * agg_df['capex'] / agg_df['sales'], 0)

# For firms that report R&D
rd_df = df_all[df_all['r&d'] > 0].groupby(["sector", "start_year"])[['sales', 'r&d']].sum().reset_index()
rd_df['r&d_sales'] = np.where(rd_df['sales'] != 0, rd_df['r&d'] / rd_df['sales'], 0)

# Generate descriptive statistics
df_all.describe(include='all')

# 1.1 CEO CHANGE ANALYSIS

In [ ]:
import pandas as pd
import numpy as np
import time
import pickle

import glob
import json
from bs4 import BeautifulSoup

pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

helper_df = pd.read_csv('231120_company_id.csv')[['ciq_id','PERMID']]
helper_df.rename(columns={'PERMID': 'permid'}, inplace=True)
helper_df = helper_df[helper_df['permid']!="0"]

ceo_final = pickle.load(open("ceo_final.p", "rb" ))
ceo_final['start_year'] = pd.to_numeric(ceo_final['start_year'])
ceo_final['end_year'] = pd.to_numeric(ceo_final['end_year'], errors='ignore')

ciq_df = pickle.load(open("ciq_data_20231120.p", "rb" ))
ciq_df.rename(columns={'year': 'start_year', 'sp_adj':'sp_adj_0x'}, inplace=True)

helper_2 = pd.read_csv('231120_company_id.csv')[['ciq_id','industry_sector']]
helper_2.rename(columns={'industry_sector':'sector'}, inplace=True)

ciq_df = ciq_df[['ciq_id','name','start_year','date','mc','sp_adj_0x']].merge(helper_2, how='left', on='ciq_id')

ciq_df['sp_adj_3'] = ciq_df.groupby(['ciq_id'])['sp_adj_0x'].transform(lambda x: x.shift(3))
ciq_df['sp_adj_2'] = ciq_df.groupby(['ciq_id'])['sp_adj_0x'].transform(lambda x: x.shift(2))
ciq_df['sp_adj_1'] = ciq_df.groupby(['ciq_id'])['sp_adj_0x'].transform(lambda x: x.shift(1))
ciq_df['tsr_3'] = ciq_df['sp_adj_2'] / ciq_df['sp_adj_3'] - 1
ciq_df['tsr_2'] = ciq_df['sp_adj_1'] / ciq_df['sp_adj_2'] - 1
ciq_df['tsr_1'] = ciq_df['sp_adj_0x'] / ciq_df['sp_adj_1'] - 1

for i in range(1,6):
    label = 'sp_adj_' + str(i) + 'x'
    ciq_df[label] = ciq_df.groupby(['ciq_id'])['sp_adj_0x'].transform(lambda x: x.shift(-1*i))
    label_prev = 'sp_adj_' + str(i-1) + 'x'
    label_ts = 'tsr_'+ str(i) +'x'
    ciq_df[label_ts] = ciq_df[label] / ciq_df[label_prev] - 1

ciq_df.tail()

ind_df = pd.read_csv("industry_data_20231127.csv")
ind_df['sector'] = ind_df['industry'].map(lambda x: x[8:-9])
ind_df.loc[ind_df.sector=="Health Care", 'sector'] = "Healthcare"

ind_df.drop(['industry', 'index', 'date', 'ciq total return gross', 'check industry', 'sp', 'growth'], axis=1, inplace=True)
ind_df.rename(columns={'total return':'ind_sp_0x', 'year':'start_year', 'ciq':'ciq_id'}, inplace=True)
ind_df = ind_df[ind_df.ind_sp_0x != 0]

ind_df['ind_sp_3'] = ind_df.groupby(['sector'])['ind_sp_0x'].transform(lambda x: x.shift(3))
ind_df['ind_sp_2'] = ind_df.groupby(['sector'])['ind_sp_0x'].transform(lambda x: x.shift(2))
ind_df['ind_sp_1'] = ind_df.groupby(['sector'])['ind_sp_0x'].transform(lambda x: x.shift(1))
ind_df['ind_tsr_3'] = ind_df['ind_sp_2'] / ind_df['ind_sp_3'] - 1
ind_df['ind_tsr_2'] = ind_df['ind_sp_1'] / ind_df['ind_sp_2'] - 1
ind_df['ind_tsr_1'] = ind_df['ind_sp_0x'] / ind_df['ind_sp_1'] - 1

for i in range(1,6):
    label = 'ind_sp_' + str(i) + 'x'
    ind_df[label] = ind_df.groupby(['sector'])['ind_sp_0x'].transform(lambda x: x.shift(-1*i))
    label_prev = 'ind_sp_' + str(i-1) + 'x'
    label_ts = 'ind_tsr_'+ str(i) +'x'
    ind_df[label_ts] = ind_df[label] / ind_df[label_prev] - 1

ind_df.tail()

merge_df = ciq_df.merge(ind_df[['sector','start_year','ind_tsr_3','ind_tsr_2','ind_tsr_1','ind_tsr_1x','ind_tsr_2x','ind_tsr_3x','ind_tsr_4x','ind_tsr_5x']], how='left', on=['sector','start_year'])
merge_df.head(20)

for i in ['3','2','1','1x','2x','3x','4x','5x']:
    merge_df['tsr_delta_'+i] = merge_df['tsr_'+i] - merge_df['ind_tsr_'+i]

for i in ['3','2','1','1x','2x','3x','4x','5x']:
    merge_df['tsr_index_'+i] = (merge_df['tsr_delta_'+i] - merge_df['tsr_delta_1'])

merge_df.head(20)

df_decline = pickle.load(open('ceo_output_20231127_decline.p', 'rb'))

#df_out = df_decline[['ciq_id','start_year','ceo_bool','ceo','prev_title','ceo_1x','ceo_title_1x','name_x','cat_2','cat_1x','cat_5x']]
###Removing name - unclear how it was used
df_out = df_decline[['ciq_id','start_year','ceo_bool','ceo','prev_title','ceo_1x','ceo_title_1x','cat_2','cat_1x','cat_5x']]
df_out = df_out.merge(merge_df[['ciq_id','start_year','tsr_index_3','tsr_index_2','tsr_index_1','tsr_index_1x','tsr_index_2x','tsr_index_3x','tsr_index_4x','tsr_index_5x']], how='left', on=['ciq_id','start_year'])

df_out.describe(include='all')

df_out.to_csv('ceo_change_analysis.csv', index=False)

# 2. INVESTMENT ANALYSIS

In [ ]:
import pandas as pd
import numpy as np
import time
import pickle
import glob
import json
from bs4 import BeautifulSoup
import scipy.stats

# Setting the display options for pandas DataFrame to view more rows and columns
pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

# Loading a DataFrame from CSV to map company ids from one format to another
helper_df = pd.read_csv('231120_company_id.csv')[['ciq_id','PERMID']]
helper_df.rename(columns={'PERMID': 'permid'}, inplace=True)  # Renaming column for consistency
helper_df = helper_df[helper_df['permid']!="0"]  # Filtering out rows where permid is "0"

# Loading CEO data from a pickle file and converting years to numeric
ceo_final = pickle.load(open("ceo_final_v2.p", "rb" ))
ceo_final['start_year'] = pd.to_numeric(ceo_final['start_year'])
ceo_final['end_year'] = pd.to_numeric(ceo_final['end_year'], errors='ignore')  # Ignoring errors in conversion

# Loading another DataFrame from a pickle file and renaming columns for consistency
ciq_load = pickle.load(open("ciq_data_20231120.p", "rb" ))
ciq_load.rename(columns={'year': 'start_year', 'sp_adj':'sp_adj_0x'}, inplace=True)

# Loading another DataFrame to get industry sector information
helper_2 = pd.read_csv('231120_company_id.csv')[['ciq_id','industry_sector']]
helper_2.rename(columns={'industry_sector':'sector'}, inplace=True)

# --- Calculating Capex to Sales Ratio ---
ciq_df = ciq_load[['ciq_id','name','start_year','date','capex','r&d','sales']].merge(helper_2, how='left', on='ciq_id')  # Merging data on common 'ciq_id' to get a consolidated DataFrame
ciq_df = ciq_df[ciq_df['sales']>0]  # Filtering out rows where sales are zero to avoid division by zero
ciq_df['capex'] = ciq_df['capex']*-1  # Multiplying Capex by -1, possibly to correct sign
ciq_df['capex_sales'] = ciq_df['capex']/ciq_df['sales']  # Calculating Capex to Sales ratio
ciq_df.head()

# Aggregating Capex and Sales at the sector and start_year level to calculate industry level Capex to Sales ratio
ind_df = ciq_df.groupby(['sector','start_year'])['capex','sales'].sum().reset_index()
ind_df['ind_capex_sales'] = ind_df['capex']/ind_df['sales']  # Calculating industry level Capex to Sales ratio
ind_df.head()

# Merging the firm level and industry level Capex to Sales ratio data
capex_df = ciq_df[['ciq_id','name','start_year','sector','capex_sales']].merge(ind_df[['sector','start_year','ind_capex_sales']], how='left', on=['sector','start_year'])
capex_df['capex_sales_delta'] = capex_df['capex_sales'] - capex_df['ind_capex_sales']  # Calculating the difference between firm level and industry level Capex to Sales ratio
capex_df.capex_sales_delta = scipy.stats.mstats.winsorize(capex_df.capex_sales_delta, limits=0.04)  # Winsorizing the data to mitigate the effect of outliers

# Calculating the change in Capex to Sales ratio over different periods
for i in range(1,6):
    capex_df['capex_sales_delta_'+str(i)+'x'] = capex_df.groupby(['ciq_id'])['capex_sales_delta'].transform(lambda x: x.shift(-1*i))

# Calculating the average and standard deviation of Capex to Sales ratio change
capex_df['capex_sales_delta_avg'] = (capex_df['capex_sales_delta'] + capex_df['capex_sales_delta_1x'] + capex_df['capex_sales_delta_2x'])/3.0
capex_df['capex_sales_delta_std'] = capex_df.groupby(['start_year'])['capex_sales_delta_avg'].transform(lambda x: (x - x.mean()) / x.std())

capex_df.describe()

# --- Calculating Firm R&D to Sales Ratio ---
ciq_df = ciq_load[['ciq_id','name','start_year','date','capex','r&d','sales']].merge(helper_2, how='left', on='ciq_id')  # Repeating the merge to ensure data consistency
ciq_df = ciq_df[ciq_df['sales']>0]  # Filtering out rows where sales are zero to avoid division by zero
ciq_df = ciq_df[ciq_df['r&d']>0]  # Filtering out rows where R&D is zero
ciq_df['rd_sales'] = ciq_df['r&d']/ciq_df['sales']  # Calculating R&D to Sales ratio
ciq_df.head()

# Aggregating R&D and Sales at the sector and start_year level to calculate industry level R&D to Sales ratio
ind_df = ciq_df.groupby(['sector','start_year'])['r&d','sales'].sum().reset_index()
ind_df['ind_rd_sales'] = ind_df['r&d']/ind_df['sales']  # Calculating industry level R&D to Sales ratio
ind_df.head()

# Merging the firm level and industry level R&D to Sales ratio data
rd_df = ciq_df[['ciq_id','name','start_year','sector','rd_sales']].merge(ind_df[['sector','start_year','ind_rd_sales']], how='left', on=['sector','start_year'])
rd_df['rd_sales_delta'] = rd_df['rd_sales'] - rd_df['ind_rd_sales']  # Calculating the difference between firm level and industry level R&D to Sales ratio
rd_df.rd_sales_delta = scipy.stats.mstats.winsorize(rd_df.rd_sales_delta, limits=0.04)  # Winsorizing the data to mitigate the effect of outliers

# Calculating the change in R&D to Sales ratio over different periods
for i in range(1,6):
    rd_df['rd_sales_delta_'+str(i)+'x'] = rd_df.groupby(['ciq_id'])['rd_sales_delta'].transform(lambda x: x.shift(-1*i))

# Calculating the average of R&D to Sales ratio change
rd_df['rd_sales_delta_avg'] = (rd_df['rd_sales_delta'] + rd_df['rd_sales_delta_1x'] + rd_df['rd_sales_delta_2x'])/3.0

# Calculating the standard deviation of R&D to Sales ratio change
rd_df['rd_sales_delta_std'] = rd_df.groupby(['start_year'])['rd_sales_delta_avg'].transform(lambda x: (x - x.mean()) / x.std())

rd_df.describe()

# --- Merging into Downturn Database ---
df_decline = pickle.load(open('ceo_output_20231127_decline.p', 'rb'))
df_decline['exclude'] = np.where(df_decline['sp_adj_5x']==0, "Y","N")  # Creating an 'exclude' flag based on a condition

# Preparing the output DataFrame by selecting specific columns and merging the investment analysis data
df_out = df_decline[['ciq_id','start_year','ceo_bool','ceo','prev_title','ceo_1x','sector','exclude','cat_2','cat_1x','cat_5x','tsr_2_chg','tsr_1x_chg','tsr_5x_chg','net_longterm_avg']]
#df_out = df_decline[['ciq_id','start_year','ceo_bool','ceo','prev_title','ceo_1x','ceo_title_1x','name_x','sector','exclude','cat_2','cat_1x','cat_5x','tsr_2_chg','tsr_1x_chg','tsr_5x_chg','net_longterm_avg','net_bio_avg']]
df_out = df_out.merge(capex_df.drop(['name','sector'], axis=1), how='left', on=['ciq_id','start_year']).merge(rd_df.drop(['name','sector'], axis=1), how='left', on=['ciq_id','start_year'])

# Getting descriptive statistics of the final DataFrame
df_out.describe(include='all')

# Saving the final DataFrame to a CSV file
df_out.to_csv('investment_analysis_20231122.csv', index=False)

# 3. TSR PATHS

In [ ]:
###TAKES: ceo_final_v2.p, 231120_copmany_id.csv, ciq_data.p, ceo_final, industry_data

import pandas as pd  # Importing the pandas library for data manipulation and analysis
import numpy as np  # Importing the numpy library for numerical operations
import time  # Importing the time module to handle time-related tasks
import pickle  # Importing the pickle module for serializing and deserializing Python object structures

import glob  # Importing the glob module to find all the pathnames matching a specified pattern
import json  # Importing the json module to work with JSON data
from bs4 import BeautifulSoup  # Importing BeautifulSoup from bs4 module for web scraping tasks

# Setting display options for pandas DataFrame to display 99 rows and columns at max
pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

# Reading a CSV file to get company identifiers and renaming the 'PERMID' column to 'permid'
helper_df = pd.read_csv('231120_company_id.csv')[['ciq_id','PERMID']]
helper_df.rename(columns={'PERMID': 'permid'}, inplace=True)
# Filtering out the rows where 'permid' is not "0"
helper_df = helper_df[helper_df['permid']!="0"]

# Loading a pickled (serialized) object into a pandas DataFrame, which contains final data regarding CEOs
ceo_final = pickle.load(open("ceo_final_v2.p", "rb" ))
# Converting the 'start_year' and 'end_year' columns to numeric type
ceo_final['start_year'] = pd.to_numeric(ceo_final['start_year'])
ceo_final['end_year'] = pd.to_numeric(ceo_final['end_year'], errors='ignore')  # Ignoring any errors during conversion

# Loading another pickled object containing CIQ data into a pandas DataFrame and selecting specific columns
ciq_df = pickle.load(open("ciq_data_20231120.p", "rb" ))[['ciq_id','name','start_year','sp_adj','sales','ebitda','eps','sp','mc']]
# Renaming the 'year' column to 'start_year'
#ciq_df.rename(columns={'year': 'start_year'}, inplace=True)

# Reading another CSV file to get the industry sector information for companies
helper_2 = pd.read_csv('231120_company_id.csv')[['ciq_id','industry_sector']]
# Renaming the 'industry_sector' column to 'sector'
helper_2.rename(columns={'industry_sector':'sector'}, inplace=True)

# Merging the 'ciq_df' DataFrame with 'helper_2' DataFrame on 'ciq_id' to get the sector information for each company
ciq_df = ciq_df.merge(helper_2, how='left', on='ciq_id')

# Displaying the first 20 rows of the 'ciq_df' DataFrame
ciq_df.head(20)

# This part of code is dedicated to calculating Firm TSR (Total Shareholder Return) Contribution
"""### FIRM TSR CONTRIBUTION"""

# List of financial metrics for which the growth and contribution will be calculated
var_list = ['sp_adj','sales','ebitda','eps','sp','mc']
'''
# Renaming the columns to indicate the base year values (0x) and calculating the values for subsequent years (1x to 5x) along with growth rates
for k in var_list:
    ciq_df.rename(columns={str(k): str(k)+"_0x"}, inplace=True)  # Renaming the columns to indicate the base year values (0x)
    for i in range(1,6):
        label = str(k) + '_' + str(i) + 'x'  # Creating the label for subsequent years (1x to 5x)
        base_label = str(k)+'_0x'  # Base label indicating the base year values
        # Shifting the values for each year and calculating the growth rates
        ciq_df[label] = ciq_df.groupby(['ciq_id'])[base_label].transform(lambda x: x.shift(-1*i))
        growth_label = label + '_growth'
        ciq_df[growth_label] = np.power((ciq_df[label]/ciq_df[base_label]), (1.0/(i)))-1
'''
for k in var_list:
    ciq_df.rename(columns={str(k): str(k)+"_0x"}, inplace=True)  # Renaming the columns to indicate the base year values (0x)
    for i in range(1,6):
        label = str(k) + '_' + str(i) + 'x'  # Creating the label for subsequent years (1x to 5x)
        base_label = str(k)+'_0x'  # Base label indicating the base year values
        # Shifting the values for each year and calculating the growth rates
        ciq_df[label] = ciq_df.groupby(['ciq_id'])[base_label].transform(lambda x: x.shift(-1*i))
        ciq_df[label] = pd.to_numeric(ciq_df[label], errors='coerce')
        ciq_df[base_label] = pd.to_numeric(ciq_df[base_label], errors='coerce')
        growth_label = label + '_growth'
        ciq_df[growth_label] = np.power((ciq_df[label]/ciq_df[base_label]), (1.0/(i)))-1


# Displaying the last few rows of the 'ciq_df' DataFrame
ciq_df.tail()

# This section calculates the contributions of sales, earnings, cash, and price to TSR for 1x, 2x, 3x, 4x, and 5x (1 year, 2 years, ... up to 5 years)

# CALCULATE CONTRIBUTIONS FOR 1 YEAR (1x)
ciq_df['sales_1x_contrib'] = ciq_df['sales_1x_growth']  # Sales contribution is simply the growth rate of sales
ciq_df['earnings_1x_contrib'] = ciq_df['ebitda_1x_growth'] - ciq_df['sales_1x_contrib']  # Earnings contribution is the growth rate of EBITDA minus sales contribution
ciq_df['cash_1x_contrib'] = ciq_df['sp_adj_1x_growth'] - ciq_df['sp_1x_growth']  # Cash contribution is the growth rate of adjusted share price minus share price growth

# Several corrections are made to ensure the accuracy and reasonableness of the calculated contributions
# If EPS data is missing, EBITDA growth is used instead for earnings contribution
ciq_df['earnings_1x_contrib'] = np.where( (np.isnan(ciq_df['earnings_1x_contrib'])), ciq_df['eps_1x_growth'] - ciq_df['sales_1x_contrib'], ciq_df['earnings_1x_contrib'])
# If earnings switch from negative to positive, the total TSR growth is used as the earnings contribution
ciq_df['earnings_1x_contrib'] = np.where( ((ciq_df['ebitda_0x'] < 0) & (ciq_df['ebitda_1x'] > 0)), ciq_df['sp_adj_1x_growth'], ciq_df['earnings_1x_contrib'])
ciq_df['earnings_1x_contrib'] = np.where( ((ciq_df['ebitda_0x'] > 0) & (ciq_df['ebitda_1x'] < 0)), ciq_df['sp_adj_1x_growth'], ciq_df['earnings_1x_contrib'])
# If earnings are still negative, the growth rate is inverted
ciq_df['earnings_1x_contrib'] = np.where( (ciq_df['ebitda_1x'] < 0), -1*ciq_df['earnings_1x_contrib'], ciq_df['earnings_1x_contrib'])
# The earnings contribution is capped at 100% to prevent unreasonable values (e.g., 88x growth)
ciq_df['earnings_1x_contrib'] = np.where( (ciq_df['earnings_1x_contrib'] < -1), -1, ciq_df['earnings_1x_contrib'])
ciq_df['earnings_1x_contrib'] = np.where( (ciq_df['earnings_1x_contrib'] > 1), 1, ciq_df['earnings_1x_contrib'])

# Calculating the price contribution as the residual of TSR growth after accounting for sales, earnings, and cash contributions
ciq_df['price_1x_contrib'] = ciq_df['sp_adj_1x_growth'] - ciq_df['sales_1x_contrib'] - ciq_df['earnings_1x_contrib'] - ciq_df['cash_1x_contrib']

# Similar calculations are repeated for 2 years (2x), 3 years (3x), 4 years (4x), and 5 years (5x)

# CALCULATE CONTRIBUTIONS FOR 2 YEARS (2x)
ciq_df['sales_2x_contrib'] = ciq_df['sales_2x_growth']
ciq_df['earnings_2x_contrib'] = ciq_df['ebitda_2x_growth'] - ciq_df['sales_2x_contrib']
ciq_df['cash_2x_contrib'] = ciq_df['sp_adj_2x_growth'] - ciq_df['sp_2x_growth']
# Corrections
ciq_df['earnings_2x_contrib'] = np.where( (np.isnan(ciq_df['earnings_2x_contrib'])), ciq_df['eps_2x_growth'] - ciq_df['sales_2x_contrib'], ciq_df['earnings_2x_contrib'])
ciq_df['earnings_2x_contrib'] = np.where( ((ciq_df['ebitda_0x'] < 0) & (ciq_df['ebitda_2x'] > 0)), ciq_df['sp_adj_2x_growth'], ciq_df['earnings_2x_contrib'])
ciq_df['earnings_2x_contrib'] = np.where( ((ciq_df['ebitda_0x'] > 0) & (ciq_df['ebitda_2x'] < 0)), ciq_df['sp_adj_2x_growth'], ciq_df['earnings_2x_contrib'])
ciq_df['earnings_2x_contrib'] = np.where( (ciq_df['ebitda_2x'] < 0), -1*ciq_df['earnings_2x_contrib'], ciq_df['earnings_2x_contrib'])
ciq_df['earnings_2x_contrib'] = np.where( (ciq_df['earnings_2x_contrib'] < -1), -1, ciq_df['earnings_2x_contrib'])
ciq_df['earnings_2x_contrib'] = np.where( (ciq_df['earnings_2x_contrib'] > 1), 1, ciq_df['earnings_2x_contrib'])
ciq_df['price_2x_contrib'] = ciq_df['sp_adj_2x_growth'] - ciq_df['sales_2x_contrib'] - ciq_df['earnings_2x_contrib'] - ciq_df['cash_2x_contrib']

# CALCULATE CONTRIBUTIONS FOR 3 YEARS (3x)
ciq_df['sales_3x_contrib'] = ciq_df['sales_3x_growth']
ciq_df['earnings_3x_contrib'] = ciq_df['ebitda_3x_growth'] - ciq_df['sales_3x_contrib']
ciq_df['cash_3x_contrib'] = ciq_df['sp_adj_3x_growth'] - ciq_df['sp_3x_growth']
# Corrections
ciq_df['earnings_3x_contrib'] = np.where( (np.isnan(ciq_df['earnings_3x_contrib'])), ciq_df['eps_3x_growth'] - ciq_df['sales_3x_contrib'], ciq_df['earnings_3x_contrib'])
ciq_df['earnings_3x_contrib'] = np.where( ((ciq_df['ebitda_0x'] < 0) & (ciq_df['ebitda_3x'] > 0)), ciq_df['sp_adj_3x_growth'], ciq_df['earnings_3x_contrib'])
ciq_df['earnings_3x_contrib'] = np.where( ((ciq_df['ebitda_0x'] > 0) & (ciq_df['ebitda_3x'] < 0)), ciq_df['sp_adj_3x_growth'], ciq_df['earnings_3x_contrib'])
ciq_df['earnings_3x_contrib'] = np.where( (ciq_df['ebitda_3x'] < 0), -1*ciq_df['earnings_3x_contrib'], ciq_df['earnings_3x_contrib'])
ciq_df['earnings_3x_contrib'] = np.where( (ciq_df['earnings_3x_contrib'] < -1), -1, ciq_df['earnings_3x_contrib'])
ciq_df['earnings_3x_contrib'] = np.where( (ciq_df['earnings_3x_contrib'] > 1), 1, ciq_df['earnings_3x_contrib'])
ciq_df['price_3x_contrib'] = ciq_df['sp_adj_3x_growth'] - ciq_df['sales_3x_contrib'] - ciq_df['earnings_3x_contrib'] - ciq_df['cash_3x_contrib']

# * 4x *

# Calculating sales contribution for the 4th period by simply assigning the growth value
ciq_df['sales_4x_contrib'] = ciq_df['sales_4x_growth']

# Calculating earnings contribution for the 4th period by subtracting sales contribution from ebitda growth
ciq_df['earnings_4x_contrib'] = ciq_df['ebitda_4x_growth'] - ciq_df['sales_4x_contrib']

# Calculating cash contribution for the 4th period by subtracting the share price growth from adjusted share price growth
ciq_df['cash_4x_contrib'] = ciq_df['sp_adj_4x_growth'] - ciq_df['sp_4x_growth']

# corrections:
# if earnings is NaN, use ebitda
# Updating earnings contribution for the 4th period based on the presence of earnings data
ciq_df['earnings_4x_contrib'] = np.where((np.isnan(ciq_df['earnings_4x_contrib'])), ciq_df['eps_4x_growth'] - ciq_df['sales_4x_contrib'], ciq_df['earnings_4x_contrib'])

# if earnings go from negative to positive, set contribution = total TSR growth:
# Updating earnings contribution for the 4th period based on the transition of earnings from negative to positive
ciq_df['earnings_4x_contrib'] = np.where(((ciq_df['ebitda_0x'] < 0) & (ciq_df['ebitda_4x'] > 0)), ciq_df['sp_adj_4x_growth'], ciq_df['earnings_4x_contrib'])
ciq_df['earnings_4x_contrib'] = np.where(((ciq_df['ebitda_0x'] > 0) & (ciq_df['ebitda_4x'] < 0)), ciq_df['sp_adj_4x_growth'], ciq_df['earnings_4x_contrib'])

# if earnings are still negative, invert growth rate:
# Inverting the growth rate for the 4th period in case earnings are still negative
ciq_df['earnings_4x_contrib'] = np.where((ciq_df['ebitda_4x'] < 0), -1 * ciq_df['earnings_4x_contrib'], ciq_df['earnings_4x_contrib'])

# cap earnings contribution at 100% - arbitrary but need to cut off the 88x growth contribution
# Capping earnings contribution for the 4th period at 100% to handle extreme growth contribution
ciq_df['earnings_4x_contrib'] = np.where((ciq_df['earnings_4x_contrib'] < -1), -1, ciq_df['earnings_4x_contrib'])
ciq_df['earnings_4x_contrib'] = np.where((ciq_df['earnings_4x_contrib'] > 1), 1, ciq_df['earnings_4x_contrib'])

# Calculating price contribution for the 4th period by subtracting other contributions from the adjusted share price growth
ciq_df['price_4x_contrib'] = ciq_df['sp_adj_4x_growth'] - ciq_df['sales_4x_contrib'] - ciq_df['earnings_4x_contrib'] - ciq_df['cash_4x_contrib']


# * 5x *

# The following block of code for the 5th period is similar to the 4th period,
# with calculations done for sales, earnings, cash, and price contributions for the 5th period.
# The logic for corrections remains the same, tailored for the 5th period data.

ciq_df['sales_5x_contrib'] = ciq_df['sales_5x_growth']
ciq_df['earnings_5x_contrib'] = ciq_df['ebitda_5x_growth'] - ciq_df['sales_5x_contrib']
ciq_df['cash_5x_contrib'] = ciq_df['sp_adj_5x_growth'] - ciq_df['sp_5x_growth']

# corrections:
ciq_df['earnings_5x_contrib'] = np.where((np.isnan(ciq_df['earnings_5x_contrib'])), ciq_df['eps_5x_growth'] - ciq_df['sales_5x_contrib'], ciq_df['earnings_5x_contrib'])
ciq_df['earnings_5x_contrib'] = np.where(((ciq_df['ebitda_0x'] < 0) & (ciq_df['ebitda_5x'] > 0)), ciq_df['sp_adj_5x_growth'], ciq_df['earnings_5x_contrib'])
ciq_df['earnings_5x_contrib'] = np.where(((ciq_df['ebitda_0x'] > 0) & (ciq_df['ebitda_5x'] < 0)), ciq_df['sp_adj_5x_growth'], ciq_df['earnings_5x_contrib'])
ciq_df['earnings_5x_contrib'] = np.where((ciq_df['ebitda_5x'] < 0), -1 * ciq_df['earnings_5x_contrib'], ciq_df['earnings_5x_contrib'])
ciq_df['earnings_5x_contrib'] = np.where((ciq_df['earnings_5x_contrib'] < -1), -1, ciq_df['earnings_5x_contrib'])
ciq_df['earnings_5x_contrib'] = np.where((ciq_df['earnings_5x_contrib'] > 1), 1, ciq_df['earnings_5x_contrib'])

ciq_df['price_5x_contrib'] = ciq_df['sp_adj_5x_growth'] - ciq_df['sales_5x_contrib'] - ciq_df['earnings_5x_contrib'] - ciq_df['cash_5x_contrib']

# The commented code line below is likely used for debugging or verification, to output the current state of the DataFrame to a CSV file.
#ciq_df.to_csv('tsr_testing.csv')

"""### INDUSTRY TSR:"""

# Loading industry data from a CSV file
ind_df = pd.read_csv("industry_data_20231127.csv")

# Extracting sector information from the industry column by removing the first 8 and last 9 characters from each entry
ind_df['sector'] = ind_df['industry'].map(lambda x: x[8:-9])

# Correcting sector name for "Health Care" to "Healthcare"
ind_df.loc[ind_df.sector=="Health Care", 'sector'] = "Healthcare"

# Dropping unnecessary columns to retain only the essential information for the analysis
ind_df.drop(['industry', 'index', 'date', 'ciq total return gross', 'check industry', 'sp', 'growth'], axis=1, inplace=True)

# Renaming columns for better clarity and consistency
ind_df.rename(columns={'total return':'ind_sp_0x', 'year':'start_year', 'ciq':'ciq_id'}, inplace=True)

# Filtering out entries with zero total share return to avoid distortions in the analysis
ind_df = ind_df[ind_df.ind_sp_0x != 0]

# The following block of code calculates the industry total share return (TSR) for different periods (1 to 5 years).
# It shifts the initial total share return values for each sector to calculate the TSR for subsequent periods.
# The TSR growth rate for each period is also calculated.

for i in range(1,6):
    label = 'ind_sp_' + str(i) + 'x'  # Label for the total share return value for the current period
    ind_df[label] = ind_df.groupby(['sector'])['ind_sp_0x'].transform(lambda x: x.shift(-1*i))  # Shifting the initial TSR values
    label_ts = 'ind_tsr_'+ str(i) +'x'  # Label for the TSR growth rate for the current period
    ind_df[label_ts] = np.power((ind_df[label]/ind_df['ind_sp_0x']), (1.0/(i)))-1  # Calculating the TSR growth rate

# Displaying the last 10 rows of the DataFrame for verification
ind_df.tail(10)

# Loading a DataFrame from a pickled file, which possibly contains the aggregate data for all firms
df_all = pickle.load(open("ceo_final.p", "rb"))

# The following block of code calculates the total earnings and cash-adjusted market cap for each firm.
# Shares are calculated by dividing the market cap by the share price.
# Total earnings are calculated by multiplying earnings per share (EPS) by the number of shares.
# Earnings are set to zero if they are negative (as negative earnings are not considered in the analysis).
# Cash-adjusted market cap is calculated by multiplying the adjusted share price by the number of shares.

# Ensure the correct columns are of numeric type
cols_to_convert = ['sp', 'sp_adj', 'sales', 'ebitda', 'eps', 'capex', 'mc', 'r&d', 'restructuring']
for col in cols_to_convert:
    df_all[col] = pd.to_numeric(df_all[col], errors='coerce')

# calculate total earnings, cash-adjusted market cap:
df_all['shares'] = df_all['mc'] / df_all['sp']  # Calculating the number of shares
df_all['earnings_tot'] = df_all['eps'] * df_all['shares']  # Calculating total earnings
df_all['earnings_tot'] = np.where(df_all['earnings_tot'] < 0, 0, df_all['earnings_tot'])  # Setting negative earnings to zero
df_all['mc_adj'] = df_all['sp_adj'] * df_all['shares']  # Calculating cash-adjusted market cap

# Filling NaN values with zeros for the specified columns
df_all[['shares','earnings_tot','mc_adj']] = df_all[['shares','earnings_tot','mc_adj']].fillna(value=0)

# The commented code line below is likely used for debugging or verification, to view specific columns of the DataFrame.
#df_all[['sp','sp_adj','eps','mc','mc_adj','earnings_tot','shares']].head(20)

# The following block of code calculates industry aggregates for capital expenditure (capex), sales, total earnings, market cap, and cash-adjusted market cap.
# The data is grouped by sector and start year, and the sum of the specified columns is calculated for each group.

# calculate industry aggregates:
# Grouping data by sector and start year, then summing up the values of specified columns for each group
agg_df = df_all[df_all['sp']>0].groupby(["sector", "start_year"])[['capex','sales','earnings_tot','mc','mc_adj']].sum().reset_index()

# Displaying the first 10 rows of the aggregated data for verification
agg_df.head(10)

# Merging the aggregated data with industry total share return (TSR) data based on sector and start year
agg_df = agg_df.merge(ind_df[['sector','start_year','ind_tsr_1x','ind_tsr_2x','ind_tsr_3x','ind_tsr_4x','ind_tsr_5x']], how='left', on=['sector','start_year'])
agg_df.head()

# Renaming columns for better clarity and consistency
agg_df.rename(columns={'sales':'ind_sales', 'earnings_tot':'ind_earnings','mc':'ind_mc','mc_adj':'ind_mc_adj'}, inplace=True)

# List of financial metrics to be analyzed
var_list = ['ind_mc','ind_mc_adj','ind_sales','ind_earnings']

# Iterating through each financial metric in the list
for k in var_list:
    # Renaming the columns to append "_0x" at the end, indicating the initial value of the metric
    agg_df.rename(columns={str(k): str(k)+"_0x"}, inplace=True)
    # Iterating through a range of 1 to 6 to calculate the value and growth rate of each metric for different periods (1 to 5 years)
    for i in range(1,6):
        # Constructing the column label for the value of the metric for the current period
        label = str(k) + '_' + str(i) + 'x'
        # Constructing the column label for the initial value of the metric
        base_label = str(k)+'_0x'
        # Shifting the initial values of the metric to calculate the value of the metric for the current period
        agg_df[label] = agg_df.groupby(['sector'])[base_label].transform(lambda x: x.shift(-1*i))
        # Constructing the column label for the growth rate of the metric for the current period
        growth_label = label + '_growth'
        # Calculating the growth rate of the metric for the current period
        agg_df[growth_label] = np.power((agg_df[label]/agg_df[base_label]), (1.0/(i)))-1
# The commented code line below is likely used for debugging or verification, to view the last 10 rows of the DataFrame.
#agg_df.tail(10)

# The following block of code calculates the contributions of sales, earnings, cash, and price for the 1st period.
# It assigns the growth rates directly for sales and calculates the contributions of earnings, cash, and price based on other financial metrics.
# Corrections are made to the earnings contribution to handle specific scenarios, such as capping the earnings contribution at 50%.

agg_df['ind_sales_1x_contrib'] = agg_df['ind_sales_1x_growth']
agg_df['ind_earnings_1x_contrib'] = agg_df['ind_earnings_1x_growth'] - agg_df['ind_sales_1x_contrib']
agg_df['ind_earnings_1x_contrib'] = np.where(agg_df['ind_earnings_1x_contrib']>0.5, 0.5, agg_df['ind_earnings_1x_contrib'])
agg_df['ind_earnings_1x_contrib'] = np.where(agg_df['ind_earnings_1x_contrib']<-0.5, -0.5, agg_df['ind_earnings_1x_contrib'])  # cap earnings contribution at 50% - arbitrary but need to cut off the 6x growth contribution
agg_df['ind_cash_1x_contrib'] = agg_df['ind_mc_adj_1x_growth'] - agg_df['ind_mc_1x_growth']
agg_df['ind_price_1x_contrib'] = agg_df['ind_mc_adj_1x_growth'] - agg_df['ind_sales_1x_contrib'] - agg_df['ind_earnings_1x_contrib'] - agg_df['ind_cash_1x_contrib']

# Distributing the difference between the industry TSR and cash-adjusted market cap growth equally among the four contributions for the 1st period
for i in ['ind_sales_1x_contrib','ind_earnings_1x_contrib','ind_cash_1x_contrib','ind_price_1x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_1x']-agg_df['ind_mc_adj_1x_growth'])/4.0

# The following blocks of code for the 2nd to 5th periods are similar to the block of code for the 1st period,
# with calculations done for sales, earnings, cash, and price contributions for each period.
# The logic for corrections and distributing the difference remains the same, tailored for each period's data.

agg_df['ind_sales_2x_contrib'] = agg_df['ind_sales_2x_growth']
agg_df['ind_earnings_2x_contrib'] = agg_df['ind_earnings_2x_growth'] - agg_df['ind_sales_2x_contrib']
agg_df['ind_earnings_2x_contrib'] = np.where(agg_df['ind_earnings_2x_contrib']>0.5, 0.5, agg_df['ind_earnings_2x_contrib'])
agg_df['ind_earnings_2x_contrib'] = np.where(agg_df['ind_earnings_2x_contrib']<-0.5, -0.5, agg_df['ind_earnings_2x_contrib'])  # cap earnings contribution at 50% - arbitrary but need to cut off the 6x growth contribution
agg_df['ind_cash_2x_contrib'] = agg_df['ind_mc_adj_2x_growth'] - agg_df['ind_mc_2x_growth']
agg_df['ind_price_2x_contrib'] = agg_df['ind_mc_adj_2x_growth'] - agg_df['ind_sales_2x_contrib'] - agg_df['ind_earnings_2x_contrib'] - agg_df['ind_cash_2x_contrib']

for i in ['ind_sales_2x_contrib','ind_earnings_2x_contrib','ind_cash_2x_contrib','ind_price_2x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_2x']-agg_df['ind_mc_adj_2x_growth'])/4.0

agg_df['ind_sales_3x_contrib'] = agg_df['ind_sales_3x_growth']
agg_df['ind_earnings_3x_contrib'] = agg_df['ind_earnings_3x_growth'] - agg_df['ind_sales_3x_contrib']
agg_df['ind_earnings_3x_contrib'] = np.where(agg_df['ind_earnings_3x_contrib']>0.5, 0.5, agg_df['ind_earnings_3x_contrib'])
agg_df['ind_earnings_3x_contrib'] = np.where(agg_df['ind_earnings_3x_contrib']<-0.5, -0.5, agg_df['ind_earnings_3x_contrib'])  # cap earnings contribution at 50% - arbitrary but need to cut off the 6x growth contribution
agg_df['ind_cash_3x_contrib'] = agg_df['ind_mc_adj_3x_growth'] - agg_df['ind_mc_3x_growth']
agg_df['ind_price_3x_contrib'] = agg_df['ind_mc_adj_3x_growth'] - agg_df['ind_sales_3x_contrib'] - agg_df['ind_earnings_3x_contrib'] - agg_df['ind_cash_3x_contrib']

for i in ['ind_sales_3x_contrib','ind_earnings_3x_contrib','ind_cash_3x_contrib','ind_price_3x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_3x']-agg_df['ind_mc_adj_3x_growth'])/4.0

agg_df['ind_sales_4x_contrib'] = agg_df['ind_sales_4x_growth']
agg_df['ind_earnings_4x_contrib'] = agg_df['ind_earnings_4x_growth'] - agg_df['ind_sales_4x_contrib']
agg_df['ind_earnings_4x_contrib'] = np.where(agg_df['ind_earnings_4x_contrib']>0.5, 0.5, agg_df['ind_earnings_4x_contrib'])
agg_df['ind_earnings_4x_contrib'] = np.where(agg_df['ind_earnings_4x_contrib']<-0.5, -0.5, agg_df['ind_earnings_4x_contrib'])  # cap earnings contribution at 50% - arbitrary but need to cut off the 6x growth contribution
agg_df['ind_cash_4x_contrib'] = agg_df['ind_mc_adj_4x_growth'] - agg_df['ind_mc_4x_growth']
agg_df['ind_price_4x_contrib'] = agg_df['ind_mc_adj_4x_growth'] - agg_df['ind_sales_4x_contrib'] - agg_df['ind_earnings_4x_contrib'] - agg_df['ind_cash_4x_contrib']

for i in ['ind_sales_4x_contrib','ind_earnings_4x_contrib','ind_cash_4x_contrib','ind_price_4x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_4x']-agg_df['ind_mc_adj_4x_growth'])/4.0

agg_df['ind_sales_5x_contrib'] = agg_df['ind_sales_5x_growth']
agg_df['ind_earnings_5x_contrib'] = agg_df['ind_earnings_5x_growth'] - agg_df['ind_sales_5x_contrib']
agg_df['ind_earnings_5x_contrib'] = np.where(agg_df['ind_earnings_5x_contrib']>0.5, 0.5, agg_df['ind_earnings_5x_contrib'])
agg_df['ind_earnings_5x_contrib'] = np.where(agg_df['ind_earnings_5x_contrib']<-0.5, -0.5, agg_df['ind_earnings_5x_contrib'])
        # cap earnings contribution at 50% - arbitrary but need to cut off the 6x growth contribution
agg_df['ind_cash_5x_contrib'] = agg_df['ind_mc_adj_5x_growth'] - agg_df['ind_mc_5x_growth']
agg_df['ind_price_5x_contrib'] = agg_df['ind_mc_adj_5x_growth'] - agg_df['ind_sales_5x_contrib'] - agg_df['ind_earnings_5x_contrib'] - agg_df['ind_cash_5x_contrib']

for i in ['ind_sales_5x_contrib','ind_earnings_5x_contrib','ind_cash_5x_contrib','ind_price_5x_contrib']:
    agg_df[i] = agg_df[i] + (agg_df['ind_tsr_5x']-agg_df['ind_mc_adj_5x_growth'])/4.0


agg_df.tail(10)

# prep for merging:
ciq_df.rename(columns={'sp_adj_1x_growth':'tsr_1x','sp_adj_2x_growth':'tsr_2x','sp_adj_3x_growth':'tsr_3x','sp_adj_4x_growth':'tsr_4x','sp_adj_5x_growth':'tsr_5x'}, inplace=True)

ciq_list = ['ciq_id','sector','start_year','tsr_1x','tsr_2x','tsr_3x','tsr_4x','tsr_5x']
for i in ciq_df.columns[70:]:
    ciq_list.append(i)
ind_list = ['sector','start_year','ind_tsr_1x','ind_tsr_2x','ind_tsr_3x','ind_tsr_4x','ind_tsr_5x']
for i in agg_df.columns[52:]:
    ind_list.append(i)


ciq_merge = ciq_df[ciq_list]
ind_merge = agg_df[ind_list]

merge_df = ciq_merge.merge(ind_merge, how='left', on=['sector','start_year'])
merge_df.head(20)

"""### MERGE INTO DOWNTURN DATABASE:"""

df_decline = pickle.load(open('ceo_output_20231127_decline.p', 'rb'))

#df_out = df_decline[['ciq_id','start_year','ceo_bool','name_x','cat_2','cat_1x','cat_5x']]
df_out = df_decline[['ciq_id','start_year','ceo_bool','cat_2','cat_1x','cat_5x']]
df_out = df_out.merge(merge_df, how='left', on=['ciq_id','start_year'])

df_out.describe(include='all')

df_out.to_csv('tsr_paths_20221127_v2.csv', index=False)

# 4. TURBULENCE ANALYSIS

In [ ]:
# Importing necessary libraries for data manipulation and analysis
import pandas as pd
import numpy as np
import time
import pickle

# Setting display options for pandas DataFrame to show 99 rows and 99 columns at maximum
pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

# Defining the industry column name for grouping and analysis; this is set as 'sector_2' to represent a particular level of industry aggregation
industry = 'sector_2'

# Reading the input data file 'turbulence_input_v3.csv' into a pandas DataFrame df
df = pd.read_csv('turbulence_input_v3.csv')

# Print statement to check the first few rows of the data
print(df.head())

# Convert all relevant columns to numeric, coercing errors (i.e., non-convertible values) to NaN
cols_to_convert = ['sp', 'sp_adj', 'sales', 'ebitda', 'eps', 'capex', 'mc', 'r&d', 'restructuring']
df[cols_to_convert] = df[cols_to_convert].apply(pd.to_numeric, errors='coerce')

# Print statement to check the data types of columns
print(df.dtypes)

# Creating 'sales_rank' column by ranking companies within each industry and year based on their sales; ranking is in descending order so higher sales get lower rank
df['sales_rank'] = df.groupby(['start_year', industry])['sales'].rank(method='min', ascending=False)

# Creating 'mc_rank' column by ranking companies within each industry and year based on their market capitalization; ranking is in descending order so higher market cap gets lower rank
df['mc_rank'] = df.groupby(['start_year', industry])['mc'].rank(method='min', ascending=False)

# Replacing ranks with NaN for companies with sales or market capitalization less or equal to zero as these values don't provide meaningful rank
df['sales_rank'] = np.where(df['sales']<=0, np.NaN, df['sales_rank'])
df['mc_rank'] = np.where(df['mc']<=0, np.NaN, df['mc_rank'])

# Check for any NaN values in the sales and mc columns
print(df[['sales', 'mc']].isnull().sum())

# Check for any NaN values in the rank columns
print(df[['sales_rank', 'mc_rank']].isnull().sum())

# Print statement to check the values in the 'sales_rank' and 'mc_rank' columns
print(df[['sales_rank', 'mc_rank']].head())

# Creating lagged rank columns 'sales_rank_1' and 'mc_rank_1' to store previous year's rank for each company
df['sales_rank_1'] = df.groupby(['ciq_id'])['sales_rank'].transform(lambda x: x.shift(1))
df['mc_rank_1'] = df.groupby(['ciq_id'])['mc_rank'].transform(lambda x: x.shift(1))

# Calculating the absolute difference in ranks between current and previous year for both sales and market capitalization
df['sales_diff'] = np.where(df['sales_rank'] < df['sales_rank_1'], df['sales_rank_1']-df['sales_rank'], df['sales_rank']-df['sales_rank_1'])
df['mc_diff'] = np.where(df['mc_rank'] < df['mc_rank_1'], df['mc_rank_1']-df['mc_rank'], df['mc_rank']-df['mc_rank_1'])

# Getting the maximum rank between current and previous year for both sales and market capitalization to use in turbulence score calculation
df['sales_max'] = np.where(df['sales_rank'] < df['sales_rank_1'], df['sales_rank_1'], df['sales_rank'])
df['mc_max'] = np.where(df['mc_rank'] < df['mc_rank_1'], df['mc_rank_1'], df['mc_rank'])

# Print statement to check for any zero or NaN values in 'sales_max' and 'mc_max' columns
print((df['sales_max'] == 0).sum(), (df['sales_max'].isna().sum()))
print((df['mc_max'] == 0).sum(), (df['mc_max'].isna().sum()))

# Calculating turbulence scores for sales and market capitalization by dividing absolute rank difference by maximum rank
df['sales_max'] = df['sales_max'].replace(0, np.nan)
df['mc_max'] = df['mc_max'].replace(0, np.nan)

df['sales_turb'] = df['sales_diff']/df['sales_max']
df['mc_turb'] = df['mc_diff']/df['mc_max']

# Print statement to check the values in the 'sales_turb' and 'mc_turb' columns
print(df[['sales_turb', 'mc_turb']].head())

# Saving intermediate results to 'turbulence_testing.csv'
df.to_csv('turbulence_testing.csv')

# Calculating mean turbulence score for sales and market capitalization per industry per year
df_sales = df.groupby(['start_year', industry])['sales_turb'].mean()
df_mc = df.groupby(['start_year', industry])['mc_turb'].mean()

# Preparing the output DataFrame df_out by merging calculated mean turbulence scores back to the original data
df_out = df[['INDEX', 'start_year', 'ciq_id', 'name', 'sector', 'sector_2', 'sector_3', 'sp', 'sp_adj', 'sales', 'mc']]\
                        .merge(df_sales.to_frame().reset_index(), how='left', on=['start_year', industry])\
                        .merge(df_mc.to_frame().reset_index(), how='left', on=['start_year', industry])

# Calculating mean and standard deviation of turbulence scores for normalization
sales_mean = df_out['sales_turb'].mean()
mc_mean = df_out['mc_turb'].mean()
sales_std = df_out['sales_turb'].std()
mc_std = df_out['mc_turb'].std()

# Calculating a combined turbulence score by normalizing and summing the sales and market cap turbulence scores
df_out['combo_turb'] = (df_out['sales_turb'] - sales_mean)/sales_std + (df_out['mc_turb']-mc_mean)/mc_std

# Creating a rolling window of size 5 to calculate 5-year average turbulence scores
df_roll = df_out.groupby('ciq_id')[['sales_turb', 'mc_turb', 'combo_turb']].rolling(5).mean()
df_roll.index = df_roll.index.droplevel()  # Dropping 'ciq_id' from index to match the structure of df_out

# Merging the 5-year average turbulence scores back to df_out
df_out['sales_turb_avg5'] = df_roll['sales_turb']
df_out['mc_turb_avg5'] = df_roll['mc_turb']

# Calculating mean and standard deviation of 5-year average turbulence scores for normalization
sales_mean_5 = df_out['sales_turb_avg5'].mean()
mc_mean_5 = df_out['mc_turb_avg5'].mean()
sales_std_5 = df_out['sales_turb_avg5'].std()
mc_std_5 = df_out['mc_turb_avg5'].std()

# Calculating a combined 5-year average turbulence score by normalizing and summing the 5-year average sales and market cap turbulence scores
df_out['combo_turb_avg5'] = (df_out['sales_turb_avg5'] - sales_mean_5)/sales_std_5 + (df_out['mc_turb_avg5']-mc_mean_5)/mc_std_5

# Viewing the first 25 rows of df_out to check the data
df_out.head(25)

# Describing df_out to get a summary of statistics
df_out.describe()

# Saving the final output to 'turbulence_analysis_v3.csv'
df_out.to_csv('turbulence_analysis_v3.csv', index=False)

# 5. TRANSFORMATION REGRESSIONS

In [ ]:
###Files needed: ceo_output_decline, ceo_tf_corrections, investment_analysis, tf_restructuring_input, company_id,

# Importing necessary libraries for data manipulation and analysis
import pandas as pd
import numpy as np
import time
import pickle
import math

# Setting display options to allow for viewing more rows and columns in the DataFrame
pd.options.display.max_rows = 99
pd.options.display.max_columns = 99

# Loading the DataFrame of 'decline' firms from a pickled file
df_decline = pickle.load(open("ceo_output_20231127_decline.p", "rb" ))

# Selecting relevant columns from the DataFrame
df_decline = df_decline[['ciq_id','start_year','name_x','sector','sp','sp_adj','sales','mc','tsr_5p','tsr_2','tsr_1x','tsr_5x','tsr_5p_delta','tsr_2_delta','tsr_1x_delta','tsr_5x_delta','tsr_2_chg','tsr_1x_chg','tsr_5x_chg','cat_2','cat_1x','cat_5x','net_longterm','net_biological','net_longterm_avg','net_bio_avg']].rename(columns={'name_x':'name'})

# Adding a column to indicate if a firm should be excluded based on a condition
df_decline['exclude'] = np.where((df_decline['tsr_5x']==-1), "Y","N")

# Generating descriptive statistics of the DataFrame
df_decline.describe(include='all')

# Loading corrected CEO data from a CSV file
ceo_change = pd.read_csv('ceo_tf_corrections.csv', index_col=False)
# Adding a boolean column to indicate if there's a CEO change
ceo_change['ceo_bool'] = np.where((pd.isnull(ceo_change['ceo']) & pd.isnull(ceo_change['ceo_1x'])), 0,1)
# Adding a boolean column to indicate if the CEO is an outsider
ceo_change['outsider_bool'] = np.where(((pd.isnull(ceo_change['prev_title']) & pd.isnull(ceo_change['ceo_title_1x'])) & (ceo_change['ceo_bool']==1)), 1,0)

# Viewing the first 20 rows of the CEO data
ceo_change.head(20)

# Loading investment data from a CSV file
df_inv = pd.read_csv('investment_analysis_20231122.csv', index_col=False)
# Selecting relevant columns from the investment data
df_inv = df_inv[['ciq_id','start_year','name_x','capex_sales_delta','capex_sales_delta_avg','capex_sales_delta_std','rd_sales_delta','rd_sales_delta_avg','rd_sales_delta_std']]

# Generating descriptive statistics of the investment data
df_inv.describe(include='all')

# Loading restructuring data from a CSV file
df_restrux = pd.read_csv('tf_restructuring_input_20170810.csv', index_col=False)

# Correcting the sign of restructuring values
df_restrux['restructuring'] = df_restrux['restructuring']*-1
df_restrux['restructuring_1x'] = df_restrux['restructuring_1x']*-1

# Calculating the restructuring to sales ratio
df_restrux['restrux_sales'] = df_restrux['restructuring']/df_restrux['sales']
df_restrux['restrux_sales_1x'] = df_restrux['restructuring_1x']/df_restrux['sales_1x']

# Adding a boolean column to indicate if a firm has undergone a transformation
df_restrux['tf_bool'] = df_restrux[['tf', 'tf_1x']].max(axis=1)
# Calculating the size of restructuring based on the maximum of the two periods
df_restrux['restrux_size'] = df_restrux[['restrux_sales', 'restrux_sales_1x']].max(axis=1)

# Calculating the mean and standard deviation of restructuring size for transformed firms
restrux_mean = df_restrux[df_restrux['tf_bool']==1]['restrux_size'].mean()
restrux_std = df_restrux[df_restrux['tf_bool']==1]['restrux_size'].std()
# Standardizing the restructuring size
df_restrux['restrux_size_std'] = df_restrux['restrux_size'].apply(lambda x: (x - restrux_mean)/restrux_std)

# Generating descriptive statistics of the restructuring data
df_restrux.describe(include='all')

# Loading other structural data from a CSV file
df_helper = pd.read_csv('231120_company_id.csv')[['ciq_id','yr_founded']]
# Converting the year founded to numeric
df_helper['yr_founded'] = pd.to_numeric(df_helper['yr_founded'])

# Generating descriptive statistics of the structural data
df_helper.describe(include='all')

# Merging all data together for final analysis
df_final = df_decline[df_decline['exclude']=='N'] \
    .merge(ceo_change[['start_year','ciq_id','ceo_bool','outsider_bool']], how='left', on=['start_year','ciq_id']) \
    .merge(df_inv.drop('name_x', axis=1), how='left', on=['start_year','ciq_id']) \
    .merge(df_restrux[['start_year','ciq_id','tf_bool','restrux_size','restrux_size_std']], how='left', on=['start_year','ciq_id']) \
    .merge(df_helper, how='left', on='ciq_id')

# Calculating the log of firm age and sales for normalization
df_final['l_age'] = np.log((df_final['start_year'] - df_final['yr_founded'] + 1))
df_final['l_sales'] = np.log((df_final['sales']))

# Calculating the mean and standard deviation for normalization
age_mean = df_final['l_age'].mean()
age_std = df_final['l_age'].std()
df_final['l_age_std'] = df_final['l_age'].apply(lambda x: (x - age_mean)/age_std)
sales_mean = df_final['l_sales'].mean()
sales_std = df_final['l_sales'].std()
df_final['l_sales_std'] = df_final['l_sales'].apply(lambda x: (x - sales_mean)/sales_std)

# Generating descriptive statistics of the final DataFrame
df_final.describe(include='all')

# Converting certain numeric variables to binary for further analysis
std_vars = ['capex_sales_delta_avg', 'rd_sales_delta_avg', 'restrux_size_std','l_age_std','l_sales_std','net_longterm']
vars_new = ['capex_bool','rd_bool','restrux_bool','age_bool','sales_bool','lt_bool']
for i in range(6):
    df_final[str(vars_new[i])] = df_final[str(std_vars[i])].apply(lambda x: x>0)



# Generating descriptive statistics of the final DataFrame after binary conversion
df_final.describe(include='all')

# Saving the final DataFrame to a CSV file for further analysis
df_final.to_csv('tf_regression_data_20170810.csv', index=False)